<a href="https://colab.research.google.com/github/hoangnecon/TestHierarchicalAttention/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U vnstock -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.8/121.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 7.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from vnstock import Vnstock #Phải tạo đối tượng
from datetime import datetime
import os

def fetch_and_combine_data(symbols, start_date, end_date):
    all_data = {}
    print(f"Bắt đầu tải dữ liệu cho các chỉ số: {symbols} từ {start_date} đến {end_date}...")

    # Khởi tạo đối tượng Vnstock
    stock_client = Vnstock()

    for symbol in symbols:
        try:
            df = stock_client.stock(symbol=symbol).quote.history(
                start=start_date,
                end=end_date
            )

            if df.empty:
                print(f"Cảnh báo: Không có dữ liệu trả về cho chỉ số {symbol}.")
                continue

            df.rename(columns={'time': 'Date', 'open': f'{symbol}_Open', 'high': f'{symbol}_High',
                               'low': f'{symbol}_Low', 'close': f'{symbol}_Close', 'volume': f'{symbol}_Volume'},
                      inplace=True)

            df['Date'] = pd.to_datetime(df['Date'])
            df.set_index('Date', inplace=True)

            all_data[symbol] = df
            print(f"Tải thành công dữ liệu cho {symbol}.")

        except Exception as e:
            print(f"Lỗi khi tải dữ liệu cho {symbol}: {e}")
            return None

    if not all_data:
        print("Lỗi: Không thể tải dữ liệu cho bất kỳ chỉ số nào.")
        return None

    combined_df = pd.concat(all_data.values(), axis=1, join='outer')
    combined_df.sort_index(inplace=True)
    combined_df.fillna(method='ffill', inplace=True)
    combined_df.dropna(inplace=True)

    print("\n✅ Hoàn tất việc tải và hợp nhất dữ liệu.")
    return combined_df

In [ ]:

SYMBOLS_TO_FETCH = ['VNINDEX', 'VN30', 'VN100']
START_DATE = '2019-01-01'
END_DATE = '2023-12-31'
OUTPUT_CSV_PATH = 'vn_indices_2019_2023.csv'


print("Bắt đầu quá trình...")
final_dataframe = fetch_and_combine_data(
    symbols=SYMBOLS_TO_FETCH,
    start_date=START_DATE,
    end_date=END_DATE
)

if final_dataframe is not None:

    final_dataframe.to_csv(OUTPUT_CSV_PATH)
    print(f"\n✅ Đã lưu thành công dữ liệu vào file: {OUTPUT_CSV_PATH}")

    print("\n5 dòng dữ liệu đầu tiên:")
    print(final_dataframe.head())

    print("\nCác file trong thư mục hiện tại của Colab:")
    !ls -lh

2025-07-10 09:37:02 - vnstock.common.data.data_explorer - INFO - Không phải là mã chứng khoán, thông tin công ty và tài chính không khả dụng.
INFO:vnstock.common.data.data_explorer:Không phải là mã chứng khoán, thông tin công ty và tài chính không khả dụng.


Bắt đầu quá trình...
Bắt đầu tải dữ liệu cho các chỉ số: ['VNINDEX', 'VN30', 'VN100'] từ 2019-01-01 đến 2023-12-31...


2025-07-10 09:37:04 - vnstock.common.data.data_explorer - INFO - Không phải là mã chứng khoán, thông tin công ty và tài chính không khả dụng.
INFO:vnstock.common.data.data_explorer:Không phải là mã chứng khoán, thông tin công ty và tài chính không khả dụng.


Tải thành công dữ liệu cho VNINDEX.


2025-07-10 09:37:05 - vnstock.common.data.data_explorer - INFO - Không phải là mã chứng khoán, thông tin công ty và tài chính không khả dụng.
INFO:vnstock.common.data.data_explorer:Không phải là mã chứng khoán, thông tin công ty và tài chính không khả dụng.


Tải thành công dữ liệu cho VN30.
Tải thành công dữ liệu cho VN100.

✅ Hoàn tất việc tải và hợp nhất dữ liệu.

✅ Đã lưu thành công dữ liệu vào file: vn_indices_2019_2023.csv

5 dòng dữ liệu đầu tiên:
            VNINDEX_Open  VNINDEX_High  VNINDEX_Low  VNINDEX_Close  \
Date                                                                 
2022-06-20       1218.37       1221.60      1180.40        1180.40   
2022-06-21       1172.98       1189.97      1162.94        1172.47   
2022-06-22       1180.11       1185.86      1162.98        1169.27   
2022-06-23       1162.06       1188.88      1162.06        1188.88   
2022-06-24       1190.82       1196.85      1185.48        1185.48   

            VNINDEX_Volume  VN30_Open  VN30_High  VN30_Low  VN30_Close  \
Date                                                                     
2022-06-20       620002900    1258.30    1262.83   1223.03     1225.56   
2022-06-21       607711900    1220.76    1240.80   1212.69     1224.54   
2022-06-22    

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.


In [ ]:
# config.py
%%writefile config.py
RAW_DATA_PATH = 'vn_indices_2019_2023.csv'
PROCESSED_DATA_PATH = 'vn_indices_processed.csv'
INDICATORS = {
    'rsi': {'window': 14},
    'macd': {'fast': 12, 'slow': 26, 'signal': 9},
    'bollinger': {'window': 20, 'std': 2}
}
TARGET_SYMBOLS = ['VNINDEX', 'VN30', 'VN100']
PRICE_COLUMN_SUFFIX = '_Close'
VOLUME_COLUMN_SUFFIX = '_Volume'

Writing config.py


In [ ]:
%%writefile feature_engineering.py
# feature_engineering.py

import pandas as pd
import numpy as np
# Import các cấu hình từ file config.py (ĐÃ SỬA: Bỏ PANDAS_FILL_METHOD)
from config import RAW_DATA_PATH, PROCESSED_DATA_PATH, INDICATORS, TARGET_SYMBOLS, PRICE_COLUMN_SUFFIX

def calculate_rsi(series, window):
    """Tính toán Chỉ số Sức mạnh Tương đối (RSI)"""
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_macd(series, fast, slow, signal):
    """Tính toán Trung bình Động Hội tụ Phân kỳ (MACD)"""
    exp1 = series.ewm(span=fast, adjust=False).mean()
    exp2 = series.ewm(span=slow, adjust=False).mean()
    macd = exp1 - exp2
    signal_line = macd.ewm(span=signal, adjust=False).mean()
    return macd, signal_line

def calculate_bollinger_bands(series, window, std):
    """Tính toán Dải Bollinger"""
    rolling_mean = series.rolling(window=window).mean()
    rolling_std = series.rolling(window=window).std()
    upper_band = rolling_mean + (rolling_std * std)
    lower_band = rolling_mean - (rolling_std * std)
    return upper_band, lower_band

def add_features(df):
    """
    Đọc dữ liệu thô, tính toán các chỉ báo kỹ thuật cho mỗi mã,
    và trả về một dataframe mới với các đặc trưng này.
    """
    print("Bắt đầu quá trình Feature Engineering...")

    for symbol in TARGET_SYMBOLS:
        print(f"-> Đang tính toán chỉ báo cho {symbol}...")
        price_col = f"{symbol}{PRICE_COLUMN_SUFFIX}"

        # --- RSI ---
        rsi_params = INDICATORS['rsi']
        df[f'{symbol}_RSI'] = calculate_rsi(df[price_col], window=rsi_params['window'])

        # --- MACD ---
        macd_params = INDICATORS['macd']
        macd, signal_line = calculate_macd(df[price_col], fast=macd_params['fast'], slow=macd_params['slow'], signal=macd_params['signal'])
        df[f'{symbol}_MACD'] = macd
        df[f'{symbol}_MACD_Signal'] = signal_line

        # --- Bollinger Bands ---
        bb_params = INDICATORS['bollinger']
        upper_band, lower_band = calculate_bollinger_bands(df[price_col], window=bb_params['window'], std=bb_params['std'])
        df[f'{symbol}_BB_Upper'] = upper_band
        df[f'{symbol}_BB_Lower'] = lower_band

    original_rows = len(df)
    df.dropna(inplace=True)
    new_rows = len(df)
    print(f"\nĐã bỏ {original_rows - new_rows} hàng có giá trị NaN sau khi tính toán đặc trưng.")

    print("\nHoàn tất Feature Engineering.")
    print("Dữ liệu mẫu sau khi xử lý:")
    print(df.head())

    return df

if __name__ == '__main__':
    try:
        raw_df = pd.read_csv(RAW_DATA_PATH, index_col='Date', parse_dates=True)
        processed_df = add_features(raw_df)
        processed_df.to_csv(PROCESSED_DATA_PATH)
        print(f"\n✅ Đã lưu thành công dữ liệu đã xử lý vào file {PROCESSED_DATA_PATH}")

    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file dữ liệu thô tại '{RAW_DATA_PATH}'.")
        print("Vui lòng chạy các ô tạo dữ liệu trước.")
    except Exception as e:
        print(f"Đã xảy ra lỗi: {e}")

Writing feature_engineering.py


In [ ]:
!python feature_engineering.py

Bắt đầu quá trình Feature Engineering...
-> Đang tính toán chỉ báo cho VNINDEX...
-> Đang tính toán chỉ báo cho VN30...
-> Đang tính toán chỉ báo cho VN100...

Đã bỏ 19 hàng có giá trị NaN sau khi tính toán đặc trưng.

Hoàn tất Feature Engineering.
Dữ liệu mẫu sau khi xử lý:
            VNINDEX_Open  VNINDEX_High  ...  VN100_BB_Upper  VN100_BB_Lower
Date                                    ...                                
2022-07-15       1183.15       1189.66  ...     1214.049565     1142.499435
2022-07-18       1184.74       1184.93  ...     1213.959151     1143.285849
2022-07-19       1176.10       1180.46  ...     1213.245691     1145.498309
2022-07-20       1188.12       1198.63  ...     1213.481968     1147.715032
2022-07-21       1195.03       1201.91  ...     1214.512315     1147.852685

[5 rows x 30 columns]

✅ Đã lưu thành công dữ liệu đã xử lý vào file vn_indices_processed.csv


In [ ]:
%%writefile config.py
# config.py
PROCESSED_DATA_PATH = 'vn_indices_processed.csv'
INDICATORS = {
    'rsi': {'window': 14},
    'macd': {'fast': 12, 'slow': 26, 'signal': 9},
    'bollinger': {'window': 20, 'std': 2}
}
TARGET_SYMBOLS = ['VNINDEX', 'VN30', 'VN100']
PRICE_COLUMN_SUFFIX = '_Close'
VOLUME_COLUMN_SUFFIX = '_Volume'

LOOKBACK_WINDOW = 30

TARGET_COLUMN = 'VNINDEX_Close'

WAVELET_FAMILY = 'db4'
WAVELET_LEVEL = 4

TEST_SET_SIZE = 0.2
BATCH_SIZE = 32

Overwriting config.py


In [ ]:
%%writefile dataset.py

import pandas as pd
import numpy as np
import pywt
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler

from config import (
    PROCESSED_DATA_PATH, LOOKBACK_WINDOW, TARGET_COLUMN,
    WAVELET_FAMILY, WAVELET_LEVEL, TEST_SET_SIZE, BATCH_SIZE
)

def create_multiscale_features(data, wavelet_family, level):
    """
    Phân rã mỗi cột trong dataframe thành các thành phần đa quy mô bằng Wavelet.
    """
    coeffs_df_list = []
    for column in data.columns:
        series = data[column].values
        coeffs = pywt.wavedec(series, wavelet_family, level=level)

        for i, c in enumerate(coeffs):
            c_padded = np.pad(c, (0, len(data) - len(c)), 'constant')
            coeff_name = f"{column}_wavelet_L{i}"
            coeffs_df_list.append(pd.DataFrame({coeff_name: c_padded}, index=data.index))

    multiscale_df = pd.concat(coeffs_df_list, axis=1)
    return multiscale_df

def create_sequences(data, target, lookback_window):
    """Tạo các chuỗi tuần tự- slide windows cho bài toán học có giám sát."""
    X, y = [], []
    for i in range(len(data) - lookback_window):
        X.append(data[i:(i + lookback_window)])
        y.append(target[i + lookback_window])
    return np.array(X), np.array(y)

class StockDataset(Dataset):
    """Lớp Dataset tùy chỉnh của PyTorch."""
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

def get_data_loaders():
    print("Bắt đầu quá trình tạo DataLoader...")
    try:
        df = pd.read_csv(PROCESSED_DATA_PATH, index_col='Date', parse_dates=True)
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file {PROCESSED_DATA_PATH}. Vui lòng chạy các bước trước.")
        return None, None, None, None

    # 2. Tạo đặc trưng đa quy mô bằng Wavelet
    print(f"-> Đang tạo đặc trưng đa quy mô với Wavelet (family: {WAVELET_FAMILY}, level: {WAVELET_LEVEL})...")
    multiscale_df = create_multiscale_features(df, WAVELET_FAMILY, WAVELET_LEVEL)

    print("-> Đang chuẩn hóa dữ liệu (scaling)...")
    feature_scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_features = feature_scaler.fit_transform(multiscale_df)

    target_scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_target = target_scaler.fit_transform(df[[TARGET_COLUMN]])

    print(f"-> Đang tạo các chuỗi tuần tự với lookback window = {LOOKBACK_WINDOW}...")
    X, y = create_sequences(scaled_features, scaled_target.flatten(), LOOKBACK_WINDOW)

    split_index = int(len(X) * (1 - TEST_SET_SIZE))
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]
    print(f"-> Kích thước tập Train: {len(X_train)} mẫu")
    print(f"-> Kích thước tập Test: {len(X_test)} mẫu")

    # 6. Tạo các đối tượng Dataset và DataLoader của PyTorch
    train_dataset = StockDataset(X_train, y_train)
    test_dataset = StockDataset(X_test, y_test)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    print("\n✅ Hoàn tất việc tạo DataLoader.")
    return train_loader, test_loader, feature_scaler, target_scaler

if __name__ == '__main__':
    # Cài đặt thư viện wavelet nếu chưa có
    try:
        import pywt
    except ImportError:
        print("Đang cài đặt thư viện PyWavelets...")
        import subprocess
        import sys
        subprocess.check_call([sys.executable, "-m", "pip", "install", "PyWavelets"])

    # Chạy hàm chính để kiểm tra
    train_loader, test_loader, _, target_scaler = get_data_loaders()

    if train_loader:
        # Kiểm tra một batch dữ liệu
        features, labels = next(iter(train_loader))
        print("\n--- Kiểm tra một batch từ DataLoader ---")
        print(f"Kích thước batch features (đầu vào): {features.shape}")
        print("==> [Số mẫu trong batch, Độ dài chuỗi (lookback), Số lượng đặc trưng]")
        print(f"Kích thước batch labels (đầu ra): {labels.shape}")

        # In ra ví dụ về giá trị đã được chuẩn hóa
        print(f"\nVí dụ giá trị label (đã chuẩn hóa): {labels[:5].flatten()}")
        # Sử dụng target_scaler để biến đổi ngược lại về giá trị gốc
        original_labels = target_scaler.inverse_transform(labels.numpy().reshape(-1, 1))
        print(f"Ví dụ giá trị label (đã giải chuẩn hóa): \n{original_labels[:5].flatten()}")

Writing dataset.py


In [ ]:
!python dataset.py

Bắt đầu quá trình tạo DataLoader...
-> Đang tạo đặc trưng đa quy mô với Wavelet (family: db4, level: 4)...
-> Đang chuẩn hóa dữ liệu (scaling)...
-> Đang tạo các chuỗi tuần tự với lookback window = 30...
-> Kích thước tập Train: 270 mẫu
-> Kích thước tập Test: 68 mẫu

✅ Hoàn tất việc tạo DataLoader.

--- Kiểm tra một batch từ DataLoader ---
Kích thước batch features (đầu vào): torch.Size([32, 30, 150])
==> [Số mẫu trong batch, Độ dài chuỗi (lookback), Số lượng đặc trưng]
Kích thước batch labels (đầu ra): torch.Size([32])

Ví dụ giá trị label (đã chuẩn hóa): tensor([0.3987, 0.6804, 0.3341, 0.7571, 0.4149])
Ví dụ giá trị label (đã giải chuẩn hóa): 
[1062.19 1168.4  1037.84 1197.33 1068.31]


In [ ]:
%%writefile config.py
# config.py

# --- Đường dẫn file ---
PROCESSED_DATA_PATH = 'vn_indices_processed.csv'
TRAINED_MODEL_PATH = 'mslstm_model.pth' #lưu model

# --- Tham số tạo Dataset ---
LOOKBACK_WINDOW = 30
TARGET_COLUMN = 'VNINDEX_Close'
TEST_SET_SIZE = 0.2
BATCH_SIZE = 32

# --- Tham số Biến đổi Wavelet ---
WAVELET_FAMILY = 'db4'
WAVELET_LEVEL = 4

# --- Tham số Kiến trúc Model ---
# Số đặc trưng gốc
# (OHLCV + 5 chỉ báo) * 3 chỉ số = 10 * 3 = 30
NUM_BASE_FEATURES = 30
# Số quy mô wavelet = level + 1 (số nhánh lstm)
NUM_SCALES = WAVELET_LEVEL + 1

# Các tham số cho lớp LSTM
LSTM_HIDDEN_UNITS = 64 # Số unit trong mỗi cell LSTM
LSTM_NUM_LAYERS = 2    # Số lớp LSTM chồng lên nhau trong mỗi nhánh

LEARNING_RATE = 0.001
NUM_EPOCHS = 50

Overwriting config.py


Bây giờ sẽ buld MS_LSTM baseline cơ bản

In [ ]:
%%writefile model.py
# model.py

import torch
import torch.nn as nn
from config import (
    NUM_BASE_FEATURES, NUM_SCALES, LSTM_HIDDEN_UNITS, LSTM_NUM_LAYERS,
    BATCH_SIZE, LOOKBACK_WINDOW
)

class MSLSTM(nn.Module):
    def __init__(self, input_feature_size, num_scales, lstm_hidden_units, lstm_num_layers):
        """  Args:
            input_feature_size (int): Số đặc trưng cho mỗi quy mô (nhánh).
            num_scales (int): Số lượng quy mô song song (số nhánh LSTM).
            lstm_hidden_units (int): Số unit ẩn trong mỗi LSTM.
            lstm_num_layers (int): Số lớp trong mỗi LSTM.
        """
        super(MSLSTM, self).__init__()

        self.num_scales = num_scales
        self.input_feature_size = input_feature_size

        # Tạo ra một danh sách các nhánh LSTM song song
        # Mỗi nhánh là một mạng LSTM độc lập
        self.lstm_branches = nn.ModuleList([
            nn.LSTM(
                input_size=input_feature_size,
                hidden_size=lstm_hidden_units,
                num_layers=lstm_num_layers,
                batch_first=True, # Định dạng input: (batch, seq_len, features)
                dropout=0.2 if lstm_num_layers > 1 else 0 # Thêm dropout nếu có nhiều lớp
            ) for _ in range(num_scales)
        ])

        # Lớp Fully Connected cuối cùng để hợp nhất kết quả từ các nhánh
        # và đưa ra dự báo cuối cùng.
        # Đầu vào của nó là tổng số unit ẩn từ tất cả các nhánh
        self.fc = nn.Linear(in_features=lstm_hidden_units * num_scales, out_features=1)

    def forward(self, x):
        """
        Định nghĩa luồng dữ liệu đi qua model.

        Args:
            x (torch.Tensor): Tensor đầu vào với shape [batch_size, lookback_window, total_features]
                               Trong đó total_features = input_feature_size * num_scales
        """
        # Danh sách để lưu trữ đầu ra từ mỗi nhánh LSTM
        lstm_outputs = []

        # Tách dữ liệu đầu vào và đưa vào từng nhánh LSTM tương ứng
        for i in range(self.num_scales):
            # Tách ra phần dữ liệu cho nhánh thứ i
            start_idx = i * self.input_feature_size
            end_idx = (i + 1) * self.input_feature_size

            branch_input = x[:, :, start_idx:end_idx]

            # Đưa dữ liệu qua nhánh LSTM thứ i
            # Chúng ta chỉ cần đầu ra cuối cùng của chuỗi (hidden_state)
            _, (hn, _) = self.lstm_branches[i](branch_input)

            # Lấy hidden state của lớp cuối cùng
            # Shape của hn là [num_layers, batch_size, hidden_units]
            # Lấy output của layer cuối cùng: hn[-1]
            lstm_outputs.append(hn[-1])

        # Concatenate các đầu ra từ tất cả các nhánh lại với nhau
        # Shape sẽ là [batch_size, lstm_hidden_units * num_scales]
        concatenated_output = torch.cat(lstm_outputs, dim=1)

        # Đưa qua lớp fully connected để có dự báo cuối cùng
        final_prediction = self.fc(concatenated_output)

        return final_prediction.squeeze() # Trả về tensor 1 chiều

if __name__ == '__main__':
    print("--- Kiểm tra kiến trúc MS-LSTM ---")

    # Dữ liệu giả lập
    batch_size = BATCH_SIZE
    lookback = LOOKBACK_WINDOW
    total_features = NUM_BASE_FEATURES * NUM_SCALES
    dummy_input = torch.randn(batch_size, lookback, total_features)

    # Khởi tạo model
    model = MSLSTM(
        input_feature_size=NUM_BASE_FEATURES,
        num_scales=NUM_SCALES,
        lstm_hidden_units=LSTM_HIDDEN_UNITS,
        lstm_num_layers=LSTM_NUM_LAYERS
    )

    # In ra kiến trúc model
    print(model)

    # Đưa dữ liệu giả qua model
    output = model(dummy_input)

    # Kiểm tra shape của đầu ra
    print(f"\nShape của input giả: {dummy_input.shape}")
    print(f"Shape của output: {output.shape}")
    print(f"==> Shape mong muốn là [{batch_size}], kết quả: {output.shape[0] == batch_size}")

Overwriting model.py


In [ ]:
!python model.py

--- Kiểm tra kiến trúc MS-LSTM ---
MSLSTM(
  (lstm_branches): ModuleList(
    (0-4): 5 x LSTM(30, 64, num_layers=2, batch_first=True, dropout=0.2)
  )
  (fc): Linear(in_features=320, out_features=1, bias=True)
)

Shape của input giả: torch.Size([32, 30, 150])
Shape của output: torch.Size([32])
==> Shape mong muốn là [32], kết quả: True


Train

In [ ]:
%%writefile train.py
# train.py

import torch
import torch.nn as nn
from tqdm import tqdm
import numpy as np

from config import (
    NUM_BASE_FEATURES, NUM_SCALES, LSTM_HIDDEN_UNITS, LSTM_NUM_LAYERS,
    LEARNING_RATE, NUM_EPOCHS, TRAINED_MODEL_PATH
)
from dataset import get_data_loaders
from model import MSLSTM

def run_training():
    """Hàm chính để thực hiện toàn bộ quá trình huấn luyện."""

    # 0. Thiết lập môi trường
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Sử dụng thiết bị: {device.upper()}")

    # 1. Tải dữ liệu (Lấy cả train và test loader)
    train_loader, test_loader, _, _ = get_data_loaders()
    if not train_loader:
        print("Dừng chương trình vì không tải được dữ liệu.")
        return

    # 2. Khởi tạo Model, Loss, Optimizer
    model = MSLSTM(
        input_feature_size=NUM_BASE_FEATURES,
        num_scales=NUM_SCALES,
        lstm_hidden_units=LSTM_HIDDEN_UNITS,
        lstm_num_layers=LSTM_NUM_LAYERS
    ).to(device)

    loss_function = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

    # 3. Vòng lặp huấn luyện
    print("\n--- Bắt đầu Huấn luyện ---")
    best_val_loss = float('inf') # Biến để lưu val_loss tốt nhất

    for epoch in range(NUM_EPOCHS):
        # --- PHA HUẤN LUYỆN (TRAINING PHASE) ---
        model.train()
        train_loss = 0.0
        # Sử dụng tqdm cho đẹp
        for features, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]"):
            features, labels = features.to(device), labels.to(device)
            outputs = model(features)
            loss = loss_function(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # --- PHA KIỂM ĐỊNH (VALIDATION PHASE) ---
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for features, labels in tqdm(test_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Valid]"):
                features, labels = features.to(device), labels.to(device)
                outputs = model(features)
                loss = loss_function(outputs, labels)
                val_loss += loss.item()

        # --- IN KẾT QUẢ VÀ LƯU MODEL TỐT NHẤT ---
        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(test_loader)

        print(f"Epoch [{epoch+1:02d}/{NUM_EPOCHS}] | Train Loss: {avg_train_loss:.6f} | Validation Loss: {avg_val_loss:.6f}")

        # Chỉ lưu model nếu validation loss của epoch này tốt hơn các epoch trước
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), TRAINED_MODEL_PATH)
            print(f"   -> Validation loss cải thiện. Đã lưu model tốt nhất vào '{TRAINED_MODEL_PATH}'")


    print(f"\n--- Huấn luyện Hoàn tất ---")
    print(f"✅ Model tốt nhất đã được lưu tại epoch có Validation Loss = {best_val_loss:.6f}")


if __name__ == '__main__':
    run_training()

Writing train.py


In [ ]:
!python train.py

Traceback (most recent call last):
  File "/content/train.py", line 12, in <module>
    from dataset import get_data_loaders
ModuleNotFoundError: No module named 'dataset'


Bây giờ sẽ chạy evaluate

In [ ]:
%%writefile evaluate.py
# evaluate.py

import torch
import numpy as np
import matplotlib.pyplot as plt

from config import (
    NUM_BASE_FEATURES, NUM_SCALES, LSTM_HIDDEN_UNITS, LSTM_NUM_LAYERS,
    TRAINED_MODEL_PATH
)
from dataset import get_data_loaders
from model import MSLSTM

def run_evaluation():
    """Hàm chính để đánh giá mô hình trên tập test."""

    # 0. Thiết lập môi trường
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Sử dụng thiết bị: {device.upper()}")

    # 1. Tải dữ liệu (chỉ cần test_loader và scaler)
    _, test_loader, _, target_scaler = get_data_loaders()
    if not test_loader:
        print("Dừng chương trình vì không tải được dữ liệu.")
        return

    # 2. Tải lại mô hình đã huấn luyện
    print(f"Đang tải mô hình từ: {TRAINED_MODEL_PATH}")
    model = MSLSTM(
        input_feature_size=NUM_BASE_FEATURES,
        num_scales=NUM_SCALES,
        lstm_hidden_units=LSTM_HIDDEN_UNITS,
        lstm_num_layers=LSTM_NUM_LAYERS
    )
    try:
        model.load_state_dict(torch.load(TRAINED_MODEL_PATH, map_location=device))
        model.to(device)
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file model tại '{TRAINED_MODEL_PATH}'.")
        print("Vui lòng chạy train.py trước.")
        return

    # 3. Đánh giá trên tập Test
    model.eval()
    predictions, actuals = [], []
    with torch.no_grad():
        for features, labels in test_loader:
            features, labels = features.to(device), labels.to(device)
            outputs = model(features)

            predictions.extend(outputs.cpu().numpy())
            actuals.extend(labels.cpu().numpy())

    # 4. Giải chuẩn hóa để so sánh
    predictions = np.array(predictions).reshape(-1, 1)
    actuals = np.array(actuals).reshape(-1, 1)

    original_predictions = target_scaler.inverse_transform(predictions)
    original_actuals = target_scaler.inverse_transform(actuals)

    # 5. Tính toán sai số và in kết quả
    mae = np.mean(np.abs(original_predictions - original_actuals))
    print(f"\n--- Kết quả Đánh giá trên Tập Test ---")
    print(f"Sai số Trung bình Tuyệt đối (MAE): {mae:.4f} (điểm VN-Index)")

    # 6. Vẽ biểu đồ so sánh
    plt.figure(figsize=(15, 6))
    plt.plot(original_actuals, label='Giá trị Thực tế (Actuals)', color='blue', marker='.', linestyle='-')
    plt.plot(original_predictions, label='Giá trị Dự đoán (Predictions)', color='red', linestyle='--')
    plt.title('So sánh Giá trị Thực tế và Dự đoán trên Tập Test')
    plt.xlabel('Ngày (trong tập Test)')
    plt.ylabel('Giá đóng cửa VN-Index')
    plt.legend()
    plt.grid(True)

    output_image_path = 'prediction_vs_actual.png'
    plt.savefig(output_image_path)
    print(f"✅ Đã lưu biểu đồ so sánh vào file '{output_image_path}'")


if __name__ == '__main__':
    run_evaluation()

Writing evaluate.py


In [ ]:
!python evaluate.py

Sử dụng thiết bị: CUDA
Bắt đầu quá trình tạo DataLoader...
-> Đang tạo đặc trưng đa quy mô với Wavelet (family: db4, level: 4)...
-> Đang chuẩn hóa dữ liệu (scaling)...
-> Đang tạo các chuỗi tuần tự với lookback window = 30...
-> Kích thước tập Train: 270 mẫu
-> Kích thước tập Test: 68 mẫu

✅ Hoàn tất việc tạo DataLoader.
Đang tải mô hình từ: mslstm_model.pth

--- Kết quả Đánh giá trên Tập Test ---
Sai số Trung bình Tuyệt đối (MAE): 20.1838 (điểm VN-Index)
✅ Đã lưu biểu đồ so sánh vào file 'prediction_vs_actual.png'


Bây giờ test tích hợp cơ chế Chú ý Đa đầu (Multi-Head Attention).

In [ ]:
%%writefile config.py
# config.py

# --- Đường dẫn file ---
PROCESSED_DATA_PATH = 'vn_indices_processed.csv'
TRAINED_MODEL_PATH = 'mslstm_attention_model.pth' # Đổi tên file model mới

# --- Tham số tạo Dataset ---
LOOKBACK_WINDOW = 30
TARGET_COLUMN = 'VNINDEX_Close'
TEST_SET_SIZE = 0.2
BATCH_SIZE = 32

# --- Tham số Biến đổi Wavelet ---
WAVELET_FAMILY = 'db4'
WAVELET_LEVEL = 4

# --- Tham số Kiến trúc Model ---
NUM_BASE_FEATURES = 30
NUM_SCALES = WAVELET_LEVEL + 1
LSTM_HIDDEN_UNITS = 64
LSTM_NUM_LAYERS = 2
# --- THAM SỐ MỚI CHO ATTENTION ---
ATTENTION_NUM_HEADS = 4 # Số "đầu" chú ý

# --- Tham số Huấn luyện ---
LEARNING_RATE = 0.001
NUM_EPOCHS = 50

Overwriting config.py


 ***MS-LSTM + Attention***

In [ ]:
%%writefile model.py
# model.py

import torch
import torch.nn as nn
from config import (
    NUM_BASE_FEATURES, NUM_SCALES, LSTM_HIDDEN_UNITS, LSTM_NUM_LAYERS,
    BATCH_SIZE, LOOKBACK_WINDOW, ATTENTION_NUM_HEADS
)

class MSLSTMAttention(nn.Module):
    def __init__(self, input_feature_size, num_scales, lstm_hidden_units, lstm_num_layers, num_heads):
        super(MSLSTMAttention, self).__init__()

        self.num_scales = num_scales
        self.input_feature_size = input_feature_size
        self.lstm_hidden_units = lstm_hidden_units

        # 1. Các nhánh LSTM song song (Không đổi)
        self.lstm_branches = nn.ModuleList([
            nn.LSTM(
                input_size=input_feature_size,
                hidden_size=lstm_hidden_units,
                num_layers=lstm_num_layers,
                batch_first=True,
                dropout=0.2 if lstm_num_layers > 1 else 0
            ) for _ in range(num_scales)
        ])

        # 2. Lớp Chú ý Đa đầu (Multi-Head Attention)
        # Nó sẽ nhận đầu vào từ các nhánh LSTM đã được ghép lại
        self.attention = nn.MultiheadAttention(
            embed_dim=lstm_hidden_units * num_scales, # Tổng kích thước đầu vào
            num_heads=num_heads,
            batch_first=True
        )

        # 3. Lớp Fully Connected cuối cùng
        # Đầu vào của nó vẫn là output từ lớp Attention
        self.fc = nn.Linear(in_features=lstm_hidden_units * num_scales, out_features=1)

    def forward(self, x):
        # Danh sách để lưu trữ đầu ra từ mỗi nhánh LSTM
        # Lần này chúng ta cần toàn bộ chuỗi đầu ra (output), không chỉ hidden state cuối
        branch_outputs = []

        for i in range(self.num_scales):
            branch_input = x[:, :, i*self.input_feature_size : (i+1)*self.input_feature_size]

            # output shape: [batch_size, seq_len, hidden_units]
            output, _ = self.lstm_branches[i](branch_input)
            branch_outputs.append(output)

        # Ghép (concatenate) các chuỗi đầu ra từ tất cả các nhánh
        # Shape: [batch_size, seq_len, lstm_hidden_units * num_scales]
        concatenated_output = torch.cat(branch_outputs, dim=2)

        # Đưa qua lớp Multi-Head Attention
        # Query, Key, Value đều là concatenated_output (self-attention)
        attention_output, _ = self.attention(concatenated_output, concatenated_output, concatenated_output)

        # Ta chỉ lấy vector cuối cùng của chuỗi sau khi qua attention để dự báo
        # Shape: [batch_size, lstm_hidden_units * num_scales]
        last_time_step_output = attention_output[:, -1, :]

        # Đưa qua lớp fully connected để có dự báo cuối cùng
        final_prediction = self.fc(last_time_step_output)

        return final_prediction.squeeze()


if __name__ == '__main__':
    # Đoạn code kiểm tra kiến trúc model mới
    print("--- Kiểm tra kiến trúc MS-LSTM + Attention ---")
    model = MSLSTMAttention(
        input_feature_size=NUM_BASE_FEATURES,
        num_scales=NUM_SCALES,
        lstm_hidden_units=LSTM_HIDDEN_UNITS,
        lstm_num_layers=LSTM_NUM_LAYERS,
        num_heads=ATTENTION_NUM_HEADS
    )
    print(model)
    dummy_input = torch.randn(BATCH_SIZE, LOOKBACK_WINDOW, NUM_BASE_FEATURES * NUM_SCALES)
    output = model(dummy_input)
    print(f"\nShape của input giả: {dummy_input.shape}")
    print(f"Shape của output: {output.shape}")
    print(f"==> Shape mong muốn là [{BATCH_SIZE}], kết quả: {output.shape[0] == BATCH_SIZE}")

Overwriting model.py


In [ ]:
!python model.py

--- Kiểm tra kiến trúc MS-LSTM + Attention ---
MSLSTMAttention(
  (lstm_branches): ModuleList(
    (0-4): 5 x LSTM(30, 64, num_layers=2, batch_first=True, dropout=0.2)
  )
  (attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=320, out_features=320, bias=True)
  )
  (fc): Linear(in_features=320, out_features=1, bias=True)
)

Shape của input giả: torch.Size([32, 30, 150])
Shape của output: torch.Size([32])
==> Shape mong muốn là [32], kết quả: True


In [ ]:
%%writefile train.py
# train.py

import torch
import torch.nn as nn
from tqdm import tqdm
import numpy as np

from config import (
    NUM_BASE_FEATURES, NUM_SCALES, LSTM_HIDDEN_UNITS, LSTM_NUM_LAYERS,
    ATTENTION_NUM_HEADS, # Thêm tham số attention
    LEARNING_RATE, NUM_EPOCHS, TRAINED_MODEL_PATH
)
# Sửa đổi: import model mới
from model import MSLSTMAttention
from dataset import get_data_loaders


def run_training():
    """Hàm chính để thực hiện toàn bộ quá trình huấn luyện."""

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Sử dụng thiết bị: {device.upper()}")

    train_loader, test_loader, _, _ = get_data_loaders()
    if not train_loader:
        print("Dừng chương trình vì không tải được dữ liệu.")
        return

    #Khởi tạo model MSLSTMAttention mới
    model = MSLSTMAttention(
        input_feature_size=NUM_BASE_FEATURES,
        num_scales=NUM_SCALES,
        lstm_hidden_units=LSTM_HIDDEN_UNITS,
        lstm_num_layers=LSTM_NUM_LAYERS,
        num_heads=ATTENTION_NUM_HEADS
    ).to(device)

    loss_function = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

    print("\n--- Bắt đầu Huấn luyện Model MS-LSTM + Attention ---")
    best_val_loss = float('inf')

    for epoch in range(NUM_EPOCHS):
        model.train()
        train_loss = 0.0
        for features, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]"):
            features, labels = features.to(device), labels.to(device)
            outputs = model(features)
            loss = loss_function(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for features, labels in tqdm(test_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Valid]"):
                features, labels = features.to(device), labels.to(device)
                outputs = model(features)
                loss = loss_function(outputs, labels)
                val_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(test_loader)

        print(f"Epoch [{epoch+1:02d}/{NUM_EPOCHS}] | Train Loss: {avg_train_loss:.6f} | Validation Loss: {avg_val_loss:.6f}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), TRAINED_MODEL_PATH)
            print(f"   -> Validation loss cải thiện. Đã lưu model tốt nhất vào '{TRAINED_MODEL_PATH}'")

    print(f"\n--- Huấn luyện Hoàn tất ---")
    print(f"✅ Model (Attention) tốt nhất đã được lưu tại epoch có Validation Loss = {best_val_loss:.6f}")

if __name__ == '__main__':
    run_training()

Overwriting train.py


In [ ]:
!python train.py

Sử dụng thiết bị: CUDA
Bắt đầu quá trình tạo DataLoader...
-> Đang tạo đặc trưng đa quy mô với Wavelet (family: db4, level: 4)...
-> Đang chuẩn hóa dữ liệu (scaling)...
-> Đang tạo các chuỗi tuần tự với lookback window = 30...
-> Kích thước tập Train: 270 mẫu
-> Kích thước tập Test: 68 mẫu

✅ Hoàn tất việc tạo DataLoader.

--- Bắt đầu Huấn luyện Model MS-LSTM + Attention ---
Epoch 1/50 [Train]: 100% 9/9 [00:00<00:00, 19.58it/s]
Epoch 1/50 [Valid]: 100% 3/3 [00:00<00:00, 56.81it/s]
Epoch [01/50] | Train Loss: 0.121739 | Validation Loss: 0.069954
   -> Validation loss cải thiện. Đã lưu model tốt nhất vào 'mslstm_attention_model.pth'
Epoch 2/50 [Train]: 100% 9/9 [00:00<00:00, 71.69it/s]
Epoch 2/50 [Valid]: 100% 3/3 [00:00<00:00, 308.01it/s]
Epoch [02/50] | Train Loss: 0.060295 | Validation Loss: 0.007198
   -> Validation loss cải thiện. Đã lưu model tốt nhất vào 'mslstm_attention_model.pth'
Epoch 3/50 [Train]: 100% 9/9 [00:00<00:00, 72.17it/s]
Epoch 3/50 [Valid]: 100% 3/3 [00:00<00:00, 31

***Evaluate sau train***

In [ ]:
%%writefile evaluate.py
# evaluate.py

import torch
import numpy as np
import matplotlib.pyplot as plt

from config import (
    NUM_BASE_FEATURES, NUM_SCALES, LSTM_HIDDEN_UNITS, LSTM_NUM_LAYERS,
    ATTENTION_NUM_HEADS,
    TRAINED_MODEL_PATH
)
#import model mới
from model import MSLSTMAttention
from dataset import get_data_loaders


def run_evaluation():
    """Hàm chính để đánh giá mô hình trên tập test."""

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Sử dụng thiết bị: {device.upper()}")

    # 1. Tải dữ liệu (chỉ cần test_loader và scaler)
    _, test_loader, _, target_scaler = get_data_loaders()
    if not test_loader:
        print("Dừng chương trình vì không tải được dữ liệu.")
        return

    # 2. Tải lại mô hình đã huấn luyện
    print(f"Đang tải mô hình từ: {TRAINED_MODEL_PATH}")
    # Sửa đổi: Khởi tạo đúng class model mới
    model = MSLSTMAttention(
        input_feature_size=NUM_BASE_FEATURES,
        num_scales=NUM_SCALES,
        lstm_hidden_units=LSTM_HIDDEN_UNITS,
        lstm_num_layers=LSTM_NUM_LAYERS,
        num_heads=ATTENTION_NUM_HEADS
    )
    try:
        model.load_state_dict(torch.load(TRAINED_MODEL_PATH, map_location=device))
        model.to(device)
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file model tại '{TRAINED_MODEL_PATH}'.")
        print("Vui lòng chạy train.py trước.")
        return

    # 3. Đánh giá trên tập Test
    model.eval()
    predictions, actuals = [], []
    with torch.no_grad():
        for features, labels in test_loader:
            features, labels = features.to(device), labels.to(device)
            outputs = model(features)

            predictions.extend(outputs.cpu().numpy())
            actuals.extend(labels.cpu().numpy())

    # 4. Giải chuẩn hóa để so sánh
    predictions = np.array(predictions).reshape(-1, 1)
    actuals = np.array(actuals).reshape(-1, 1)

    original_predictions = target_scaler.inverse_transform(predictions)
    original_actuals = target_scaler.inverse_transform(actuals)

    # 5. Tính toán sai số và in kết quả
    mae = np.mean(np.abs(original_predictions - original_actuals))
    print(f"\n--- Kết quả Đánh giá trên Tập Test (Model + Attention) ---")
    print(f"Sai số Trung bình Tuyệt đối (MAE): {mae:.4f} (điểm VN-Index)")

    # 6. Vẽ biểu đồ so sánh
    plt.figure(figsize=(15, 6))
    plt.plot(original_actuals, label='Giá trị Thực tế (Actuals)', color='blue', marker='.', linestyle='-')
    plt.plot(original_predictions, label='Giá trị Dự đoán (Predictions)', color='red', linestyle='--')
    plt.title('So sánh Giá trị Thực tế và Dự đoán (Model + Attention)')
    plt.xlabel('Ngày (trong tập Test)')
    plt.ylabel('Giá đóng cửa VN-Index')
    plt.legend()
    plt.grid(True)

    output_image_path = 'prediction_vs_actual_attention.png'
    plt.savefig(output_image_path)
    print(f"✅ Đã lưu biểu đồ so sánh vào file '{output_image_path}'")


if __name__ == '__main__':
    run_evaluation()

Overwriting evaluate.py


In [ ]:
!python evaluate.py

Sử dụng thiết bị: CUDA
Bắt đầu quá trình tạo DataLoader...
-> Đang tạo đặc trưng đa quy mô với Wavelet (family: db4, level: 4)...
-> Đang chuẩn hóa dữ liệu (scaling)...
-> Đang tạo các chuỗi tuần tự với lookback window = 30...
-> Kích thước tập Train: 270 mẫu
-> Kích thước tập Test: 68 mẫu

✅ Hoàn tất việc tạo DataLoader.
Đang tải mô hình từ: mslstm_attention_model.pth

--- Kết quả Đánh giá trên Tập Test (Model + Attention) ---
Sai số Trung bình Tuyệt đối (MAE): 20.4384 (điểm VN-Index)
✅ Đã lưu biểu đồ so sánh vào file 'prediction_vs_actual_attention.png'


# ***MAE thậm chí còn thấp hơn, bây giờ sẽ chỉnh sửa siêu tham số***

In [ ]:
%%writefile config.py
# config.py

# --- Đường dẫn file ---
PROCESSED_DATA_PATH = 'vn_indices_processed.csv'
# Đổi tên file model để lưu kết quả của lần chạy mới
TRAINED_MODEL_PATH = 'mslstm_attention_tuned.pth'

# --- Tham số tạo Dataset ---
LOOKBACK_WINDOW = 30
TARGET_COLUMN = 'VNINDEX_Close'
TEST_SET_SIZE = 0.2
BATCH_SIZE = 32

# --- Tham số Biến đổi Wavelet ---
WAVELET_FAMILY = 'db4'
WAVELET_LEVEL = 4

# --- Tham số Kiến trúc Model ---
NUM_BASE_FEATURES = 30
NUM_SCALES = WAVELET_LEVEL + 1
# --- THAY ĐỔI ---
LSTM_HIDDEN_UNITS = 128 #Tăng từ 64 lên 148 units
LSTM_NUM_LAYERS = 2
ATTENTION_NUM_HEADS = 4

# --- Tham số Huấn luyện ---
# --- THAY ĐỔI ---
LEARNING_RATE = 0.0005 # Giảm lại learning rate
NUM_EPOCHS = 100       # Tăng epoch để xem có phát huy được không

Overwriting config.py


## ***Train và evaluate lại config mới***

In [ ]:
%%writefile train.py
# train.py

import torch
import torch.nn as nn
from tqdm import tqdm
import numpy as np

from config import (
    NUM_BASE_FEATURES, NUM_SCALES, LSTM_HIDDEN_UNITS, LSTM_NUM_LAYERS,
    ATTENTION_NUM_HEADS, # Thêm tham số attention
    LEARNING_RATE, NUM_EPOCHS, TRAINED_MODEL_PATH
)
# Sửa đổi: import model mới
from model import MSLSTMAttention
from dataset import get_data_loaders


def run_training():
    """Hàm chính để thực hiện toàn bộ quá trình huấn luyện."""

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Sử dụng thiết bị: {device.upper()}")

    train_loader, test_loader, _, _ = get_data_loaders()
    if not train_loader:
        print("Dừng chương trình vì không tải được dữ liệu.")
        return

    #Khởi tạo model MSLSTMAttention mới
    model = MSLSTMAttention(
        input_feature_size=NUM_BASE_FEATURES,
        num_scales=NUM_SCALES,
        lstm_hidden_units=LSTM_HIDDEN_UNITS,
        lstm_num_layers=LSTM_NUM_LAYERS,
        num_heads=ATTENTION_NUM_HEADS
    ).to(device)

    loss_function = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

    print("\n--- Bắt đầu Huấn luyện Model MS-LSTM + Attention ---")
    best_val_loss = float('inf')

    for epoch in range(NUM_EPOCHS):
        model.train()
        train_loss = 0.0
        for features, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]"):
            features, labels = features.to(device), labels.to(device)
            outputs = model(features)
            loss = loss_function(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for features, labels in tqdm(test_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Valid]"):
                features, labels = features.to(device), labels.to(device)
                outputs = model(features)
                loss = loss_function(outputs, labels)
                val_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(test_loader)

        print(f"Epoch [{epoch+1:02d}/{NUM_EPOCHS}] | Train Loss: {avg_train_loss:.6f} | Validation Loss: {avg_val_loss:.6f}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), TRAINED_MODEL_PATH)
            print(f"   -> Validation loss cải thiện. Đã lưu model tốt nhất vào '{TRAINED_MODEL_PATH}'")

    print(f"\n--- Huấn luyện Hoàn tất ---")
    print(f"✅ Model (Attention) tốt nhất đã được lưu tại epoch có Validation Loss = {best_val_loss:.6f}")

if __name__ == '__main__':
    run_training()

Overwriting train.py


In [ ]:
!python train.py

Sử dụng thiết bị: CUDA
Bắt đầu quá trình tạo DataLoader...
-> Đang tạo đặc trưng đa quy mô với Wavelet (family: db4, level: 4)...
-> Đang chuẩn hóa dữ liệu (scaling)...
-> Đang tạo các chuỗi tuần tự với lookback window = 30...
-> Kích thước tập Train: 270 mẫu
-> Kích thước tập Test: 68 mẫu

✅ Hoàn tất việc tạo DataLoader.

--- Bắt đầu Huấn luyện Model MS-LSTM + Attention ---
Epoch 1/100 [Train]: 100% 9/9 [00:00<00:00, 26.19it/s]
Epoch 1/100 [Valid]: 100% 3/3 [00:00<00:00, 130.50it/s]
Epoch [01/100] | Train Loss: 0.105180 | Validation Loss: 0.050690
   -> Validation loss cải thiện. Đã lưu model tốt nhất vào 'mslstm_attention_tuned.pth'
Epoch 2/100 [Train]: 100% 9/9 [00:00<00:00, 85.90it/s]
Epoch 2/100 [Valid]: 100% 3/3 [00:00<00:00, 225.87it/s]
Epoch [02/100] | Train Loss: 0.056489 | Validation Loss: 0.012146
   -> Validation loss cải thiện. Đã lưu model tốt nhất vào 'mslstm_attention_tuned.pth'
Epoch 3/100 [Train]: 100% 9/9 [00:00<00:00, 86.77it/s]
Epoch 3/100 [Valid]: 100% 3/3 [00:00<

***Evalute lại***

In [ ]:
%%writefile evaluate.py
# evaluate.py

import torch
import numpy as np
import matplotlib.pyplot as plt

from config import (
    NUM_BASE_FEATURES, NUM_SCALES, LSTM_HIDDEN_UNITS, LSTM_NUM_LAYERS,
    ATTENTION_NUM_HEADS,
    TRAINED_MODEL_PATH
)
#import model mới
from model import MSLSTMAttention
from dataset import get_data_loaders


def run_evaluation():
    """Hàm chính để đánh giá mô hình trên tập test."""

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Sử dụng thiết bị: {device.upper()}")

    # 1. Tải dữ liệu (chỉ cần test_loader và scaler)
    _, test_loader, _, target_scaler = get_data_loaders()
    if not test_loader:
        print("Dừng chương trình vì không tải được dữ liệu.")
        return

    # 2. Tải lại mô hình đã huấn luyện
    print(f"Đang tải mô hình từ: {TRAINED_MODEL_PATH}")
    # Sửa đổi: Khởi tạo đúng class model mới
    model = MSLSTMAttention(
        input_feature_size=NUM_BASE_FEATURES,
        num_scales=NUM_SCALES,
        lstm_hidden_units=LSTM_HIDDEN_UNITS,
        lstm_num_layers=LSTM_NUM_LAYERS,
        num_heads=ATTENTION_NUM_HEADS
    )
    try:
        model.load_state_dict(torch.load(TRAINED_MODEL_PATH, map_location=device))
        model.to(device)
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file model tại '{TRAINED_MODEL_PATH}'.")
        print("Vui lòng chạy train.py trước.")
        return

    # 3. Đánh giá trên tập Test
    model.eval()
    predictions, actuals = [], []
    with torch.no_grad():
        for features, labels in test_loader:
            features, labels = features.to(device), labels.to(device)
            outputs = model(features)

            predictions.extend(outputs.cpu().numpy())
            actuals.extend(labels.cpu().numpy())

    # 4. Giải chuẩn hóa để so sánh
    predictions = np.array(predictions).reshape(-1, 1)
    actuals = np.array(actuals).reshape(-1, 1)

    original_predictions = target_scaler.inverse_transform(predictions)
    original_actuals = target_scaler.inverse_transform(actuals)

    # 5. Tính toán sai số và in kết quả
    mae = np.mean(np.abs(original_predictions - original_actuals))
    print(f"\n--- Kết quả Đánh giá trên Tập Test (Model + Attention) ---")
    print(f"Sai số Trung bình Tuyệt đối (MAE): {mae:.4f} (điểm VN-Index)")

    # 6. Vẽ biểu đồ so sánh
    plt.figure(figsize=(15, 6))
    plt.plot(original_actuals, label='Giá trị Thực tế (Actuals)', color='blue', marker='.', linestyle='-')
    plt.plot(original_predictions, label='Giá trị Dự đoán (Predictions)', color='red', linestyle='--')
    plt.title('So sánh Giá trị Thực tế và Dự đoán (Model + Attention)')
    plt.xlabel('Ngày (trong tập Test)')
    plt.ylabel('Giá đóng cửa VN-Index')
    plt.legend()
    plt.grid(True)

    output_image_path = 'prediction_vs_actual_attention.png'
    plt.savefig(output_image_path)
    print(f"✅ Đã lưu biểu đồ so sánh vào file '{output_image_path}'")


if __name__ == '__main__':
    run_evaluation()

Overwriting evaluate.py


In [ ]:
!python evaluate.py

Sử dụng thiết bị: CUDA
Bắt đầu quá trình tạo DataLoader...
-> Đang tạo đặc trưng đa quy mô với Wavelet (family: db4, level: 4)...
-> Đang chuẩn hóa dữ liệu (scaling)...
-> Đang tạo các chuỗi tuần tự với lookback window = 30...
-> Kích thước tập Train: 270 mẫu
-> Kích thước tập Test: 68 mẫu

✅ Hoàn tất việc tạo DataLoader.
Đang tải mô hình từ: mslstm_attention_tuned.pth

--- Kết quả Đánh giá trên Tập Test (Model + Attention) ---
Sai số Trung bình Tuyệt đối (MAE): 20.2117 (điểm VN-Index)
✅ Đã lưu biểu đồ so sánh vào file 'prediction_vs_actual_attention.png'


# ***Kết quả ra không ổn lắm, thử qua Hierachical để xem phân cấp phụ thuộc có ổn không?***

In [ ]:
%%writefile dataset.py

import pandas as pd
import numpy as np
import pywt
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler

from config import (
    PROCESSED_DATA_PATH, LOOKBACK_WINDOW, TARGET_COLUMN,
    WAVELET_FAMILY, WAVELET_LEVEL, TEST_SET_SIZE, BATCH_SIZE
)

def create_multiscale_features(data, wavelet_family, level):
    """Phân rã mỗi cột thành các thành phần đa quy mô bằng Wavelet."""
    coeffs_df_list = []
    # Dữ liệu đầu vào `data` bây giờ sẽ là 30 cột gốc
    for column in data.columns:
        series = data[column].values
        coeffs = pywt.wavedec(series, wavelet_family, level=level)
        for i, c in enumerate(coeffs):
            c_padded = np.pad(c, (0, len(data) - len(c)), 'constant')
            coeffs_df_list.append(pd.DataFrame({f"{column}_wavelet_L{i}": c_padded}, index=data.index))
    return pd.concat(coeffs_df_list, axis=1)

def create_multitask_labels(df, target_col, lookback_window):
    """Tạo ra 3 loại nhãn cho bài toán đa nhiệm."""
    price_labels = df[target_col].values
    trend_labels = (df[target_col].diff() > 0).astype(int).values
    daily_returns = df[target_col].pct_change()
    volatility_labels = daily_returns.rolling(window=lookback_window).std().values
    return price_labels, trend_labels, volatility_labels

def create_sequences(features, price_labels, trend_labels, vol_labels, lookback):
    """Tạo các chuỗi tuần tự cho cả features và 3 loại labels."""
    X, y_price, y_trend, y_vol = [], [], [], []
    for i in range(len(features) - lookback):
        X.append(features[i:(i + lookback)])
        y_price.append(price_labels[i + lookback])
        y_trend.append(trend_labels[i + lookback])
        y_vol.append(vol_labels[i + lookback])
    return np.array(X), np.array(y_price), np.array(y_trend), np.array(y_vol)

class StockDataset(Dataset):
    """Dataset tùy chỉnh cho bài toán đa nhiệm."""
    def __init__(self, features, price_lbl, trend_lbl, vol_lbl):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.price_lbl = torch.tensor(price_lbl, dtype=torch.float32)
        self.trend_lbl = torch.tensor(trend_lbl, dtype=torch.float32)
        self.vol_lbl = torch.tensor(vol_lbl, dtype=torch.float32)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return (self.features[idx],
                (self.price_lbl[idx], self.trend_lbl[idx], self.vol_lbl[idx]))

def get_data_loaders():
    """Hàm chính để tải, xử lý và tạo ra các DataLoader đa nhiệm."""
    try:
        df = pd.read_csv(PROCESSED_DATA_PATH, index_col='Date', parse_dates=True)
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file {PROCESSED_DATA_PATH}. Vui lòng chạy các bước trước.")
        return None, None, None, None

    # === LOGIC ĐÃ ĐƯỢC SỬA LẠI ===
    # 1. Tạo đặc trưng Wavelet từ TOÀN BỘ 30 cột gốc
    # df ở đây có 30 cột đặc trưng đã tính toán (OHLCV + indicators cho 3 symbols)
    multiscale_df = create_multiscale_features(df, WAVELET_FAMILY, WAVELET_LEVEL)
    # => multiscale_df bây giờ sẽ có đúng 30 * 5 = 150 cột

    # 2. Tạo các nhãn một cách riêng biệt
    price_lbl, trend_lbl, vol_lbl = create_multitask_labels(df, TARGET_COLUMN, LOOKBACK_WINDOW)

    # 3. Kết hợp features và labels vào một DataFrame để dropna đồng bộ
    labels_df = pd.DataFrame({
        'price_label': price_lbl,
        'trend_label': trend_lbl,
        'volatility_label': vol_lbl
    }, index=df.index)

    full_df = pd.concat([multiscale_df, labels_df], axis=1)
    full_df.dropna(inplace=True)

    # 4. Tách lại features và labels sau khi đã làm sạch
    final_features = full_df.drop(columns=['price_label', 'trend_label', 'volatility_label']).values
    final_price_lbl = full_df['price_label'].values
    final_trend_lbl = full_df['trend_label'].values
    final_vol_lbl = full_df['volatility_label'].values

    # 5. Chuẩn hóa dữ liệu
    feature_scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_features = feature_scaler.fit_transform(final_features)

    target_scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_price_lbl = target_scaler.fit_transform(final_price_lbl.reshape(-1, 1)).flatten()

    volatility_scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_vol_lbl = volatility_scaler.fit_transform(final_vol_lbl.reshape(-1, 1)).flatten()

    # 6. Tạo chuỗi và DataLoader như cũ
    X, y_p, y_t, y_v = create_sequences(scaled_features, scaled_price_lbl, final_trend_lbl, scaled_vol_lbl, LOOKBACK_WINDOW)
    split = int(len(X) * (1 - TEST_SET_SIZE))
    train_dataset = StockDataset(X[:split], y_p[:split], y_t[:split], y_v[:split])
    test_dataset = StockDataset(X[split:], y_p[split:], y_t[split:], y_v[split:])
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    print("✅ DataLoader đa nhiệm đã sẵn sàng.")
    return train_loader, test_loader, target_scaler, volatility_scaler

Overwriting dataset.py


In [ ]:
!python dataset.py

In [ ]:
%%writefile config.py
# config.py

# --- Đường dẫn file ---
PROCESSED_DATA_PATH = 'vn_indices_processed.csv'
# Đây là file model cuối cùng
TRAINED_MODEL_PATH = 'final_hierarchical_model.pth'

# --- Tham số tạo Dataset ---
LOOKBACK_WINDOW = 30
# Cột mục tiêu chính vẫn là VNINDEX Close
TARGET_COLUMN = 'VNINDEX_Close'
TEST_SET_SIZE = 0.2
BATCH_SIZE = 32

# --- Tham số Biến đổi Wavelet ---
WAVELET_FAMILY = 'db4'
WAVELET_LEVEL = 4

# --- Tham số Kiến trúc Model ---
NUM_BASE_FEATURES = 30
NUM_SCALES = WAVELET_LEVEL + 1
LSTM_HIDDEN_UNITS = 128
LSTM_NUM_LAYERS = 2
ATTENTION_NUM_HEADS = 4

# --- THAM SỐ MỚI CHO MULTI-TASK LEARNING ---
LOSS_WEIGHTS = {
    'price': 0.6,
    'trend': 0.3,
    'volatility': 0.1
}

# --- Tham số Huấn luyện ---
LEARNING_RATE = 0.0005
NUM_EPOCHS = 100

Overwriting config.py


In [ ]:
%%writefile model.py
# model.py

import torch
import torch.nn as nn
from config import BATCH_SIZE, LOOKBACK_WINDOW # Chỉ import những gì cần cho việc test

class FinalModel(nn.Module):
    def __init__(self, input_feature_size, num_scales, lstm_hidden_units, lstm_num_layers, num_heads):
        super(FinalModel, self).__init__()

        # SỬA LỖI: Lưu lại các tham số kiến trúc làm thuộc tính của class
        self.input_feature_size = input_feature_size
        self.num_scales = num_scales

        # --- Phần thân chung (Shared Body) ---
        self.lstm_branches = nn.ModuleList([
            nn.LSTM(input_size=self.input_feature_size, hidden_size=lstm_hidden_units,
                    num_layers=lstm_num_layers, batch_first=True, dropout=0.2 if lstm_num_layers > 1 else 0)
            for _ in range(self.num_scales)
        ])
        self.attention = nn.MultiheadAttention(embed_dim=lstm_hidden_units * self.num_scales,
                                             num_heads=num_heads, batch_first=True)

        shared_feature_dim = lstm_hidden_units * self.num_scales
        self.intermediate_layer = nn.Sequential(
            nn.Linear(shared_feature_dim, shared_feature_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.3)
        )

        # --- Các Đầu ra Đa nhiệm (Multi-Task Heads) ---
        self.price_head = nn.Linear(shared_feature_dim // 2, 1)
        self.trend_head = nn.Sequential(
            nn.Linear(shared_feature_dim // 2, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
        self.volatility_head = nn.Linear(shared_feature_dim // 2, 1)

    def forward(self, x):
        branch_outputs = []
        for i in range(self.num_scales):
            # SỬA LỖI: Dùng thuộc tính của class (self.input_feature_size) thay vì biến toàn cục
            start_idx = i * self.input_feature_size
            end_idx = (i + 1) * self.input_feature_size

            branch_input = x[:, :, start_idx:end_idx]
            output, _ = self.lstm_branches[i](branch_input)
            branch_outputs.append(output)

        concatenated_output = torch.cat(branch_outputs, dim=2)
        attention_output, _ = self.attention(concatenated_output, concatenated_output, concatenated_output)
        shared_features = self.intermediate_layer(attention_output[:, -1, :])

        price_prediction = self.price_head(shared_features)
        trend_prediction = self.trend_head(shared_features)
        volatility_prediction = self.volatility_head(shared_features)

        return price_prediction.squeeze(), trend_prediction.squeeze(), volatility_prediction.squeeze()


if __name__ == '__main__':
    # Đoạn test này cần import từ config
    from config import NUM_BASE_FEATURES, NUM_SCALES, LSTM_HIDDEN_UNITS, LSTM_NUM_LAYERS, ATTENTION_NUM_HEADS

    print("--- Kiểm tra kiến trúc Final Model (đã sửa lỗi) ---")
    model = FinalModel(
        input_feature_size=NUM_BASE_FEATURES, num_scales=NUM_SCALES,
        lstm_hidden_units=LSTM_HIDDEN_UNITS, lstm_num_layers=LSTM_NUM_LAYERS,
        num_heads=ATTENTION_NUM_HEADS
    )
    print(model)
    dummy_input = torch.randn(BATCH_SIZE, LOOKBACK_WINDOW, NUM_BASE_FEATURES * NUM_SCALES)
    price, trend, volatility = model(dummy_input)

    print(f"\nShape của input: {dummy_input.shape}")
    print(f"Shape đầu ra Price: {price.shape}")
    print(f"Shape đầu ra Trend: {trend.shape}")
    print(f"Shape đầu ra Volatility: {volatility.shape}")

Overwriting model.py


In [ ]:
!python model.py

--- Kiểm tra kiến trúc Final Model (đã sửa lỗi) ---
FinalModel(
  (lstm_branches): ModuleList(
    (0-4): 5 x LSTM(30, 128, num_layers=2, batch_first=True, dropout=0.2)
  )
  (attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=640, out_features=640, bias=True)
  )
  (intermediate_layer): Sequential(
    (0): Linear(in_features=640, out_features=320, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
  )
  (price_head): Linear(in_features=320, out_features=1, bias=True)
  (trend_head): Sequential(
    (0): Linear(in_features=320, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=1, bias=True)
  )
  (volatility_head): Linear(in_features=320, out_features=1, bias=True)
)

Shape của input: torch.Size([32, 30, 150])
Shape đầu ra Price: torch.Size([32])
Shape đầu ra Trend: torch.Size([32])
Shape đầu ra Volatility: torch.Size([32])


In [ ]:
%%writefile train.py
# train.py

import torch
import torch.nn as nn
from tqdm import tqdm

from config import (
    NUM_BASE_FEATURES, NUM_SCALES, LSTM_HIDDEN_UNITS, LSTM_NUM_LAYERS,
    ATTENTION_NUM_HEADS, LOSS_WEIGHTS,
    LEARNING_RATE, NUM_EPOCHS, TRAINED_MODEL_PATH
)
from model import FinalModel
from dataset import get_data_loaders

def run_training():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Sử dụng thiết bị: {device.upper()}")

    train_loader, test_loader, _, _ = get_data_loaders()
    if not train_loader: return

    model = FinalModel(
        input_feature_size=NUM_BASE_FEATURES, num_scales=NUM_SCALES,
        lstm_hidden_units=LSTM_HIDDEN_UNITS, lstm_num_layers=LSTM_NUM_LAYERS,
        num_heads=ATTENTION_NUM_HEADS
    ).to(device)

    # Định nghĩa các hàm loss cho từng nhiệm vụ
    price_loss_fn = nn.MSELoss() # Cho giá (hồi quy)
    # Dùng BCEWithLogitsLoss cho độ ổn định số học, nó đã tích hợp sẵn Sigmoid
    trend_loss_fn = nn.BCEWithLogitsLoss() # Cho xu hướng (phân loại)
    vol_loss_fn = nn.MSELoss() # Cho độ biến động (hồi quy)

    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

    print("\n--- Bắt đầu Huấn luyện Model Cuối cùng (Hierarchical + Multi-Task) ---")
    best_val_loss = float('inf')

    for epoch in range(NUM_EPOCHS):
        # --- PHA HUẤN LUYỆN ---
        model.train()
        total_train_loss = 0
        for features, (price_lbl, trend_lbl, vol_lbl) in tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]"):
            features = features.to(device)
            price_lbl, trend_lbl, vol_lbl = price_lbl.to(device), trend_lbl.to(device), vol_lbl.to(device)

            # Lấy 3 đầu ra từ model
            price_pred, trend_pred, vol_pred = model(features)

            # Tính toán loss cho từng nhiệm vụ
            loss_p = price_loss_fn(price_pred, price_lbl)
            loss_t = trend_loss_fn(trend_pred, trend_lbl)
            loss_v = vol_loss_fn(vol_pred, vol_lbl)

            # Tính loss tổng hợp dựa trên trọng số
            total_loss = (LOSS_WEIGHTS['price'] * loss_p +
                          LOSS_WEIGHTS['trend'] * loss_t +
                          LOSS_WEIGHTS['volatility'] * loss_v)

            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()
            total_train_loss += total_loss.item()

        # --- PHA KIỂM ĐỊNH ---
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for features, (price_lbl, trend_lbl, vol_lbl) in tqdm(test_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Valid]"):
                features = features.to(device)
                price_lbl, trend_lbl, vol_lbl = price_lbl.to(device), trend_lbl.to(device), vol_lbl.to(device)

                price_pred, trend_pred, vol_pred = model(features)

                loss_p = price_loss_fn(price_pred, price_lbl)
                loss_t = trend_loss_fn(trend_pred, trend_lbl)
                loss_v = vol_loss_fn(vol_pred, vol_lbl)

                total_loss = (LOSS_WEIGHTS['price'] * loss_p +
                              LOSS_WEIGHTS['trend'] * loss_t +
                              LOSS_WEIGHTS['volatility'] * loss_v)
                total_val_loss += total_loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        avg_val_loss = total_val_loss / len(test_loader)

        print(f"Epoch [{epoch+1:02d}/{NUM_EPOCHS}] | Train Loss: {avg_train_loss:.6f} | Validation Loss: {avg_val_loss:.6f}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), TRAINED_MODEL_PATH)
            print(f"   -> Validation loss cải thiện. Đã lưu model tốt nhất vào '{TRAINED_MODEL_PATH}'")

    print(f"\n--- Huấn luyện Hoàn tất ---")
    print(f"✅ Model cuối cùng đã được lưu tại epoch có Validation Loss = {best_val_loss:.6f}")

if __name__ == '__main__':
    run_training()

Overwriting train.py


In [ ]:
!python train.py

Sử dụng thiết bị: CUDA
✅ DataLoader đa nhiệm đã sẵn sàng.

--- Bắt đầu Huấn luyện Model Cuối cùng (Hierarchical + Multi-Task) ---
Epoch 1/100 [Train]: 100% 8/8 [00:00<00:00, 13.22it/s]
Epoch 1/100 [Valid]: 100% 2/2 [00:00<00:00, 142.25it/s]
Epoch [01/100] | Train Loss: 0.287770 | Validation Loss: 0.220904
   -> Validation loss cải thiện. Đã lưu model tốt nhất vào 'final_hierarchical_model.pth'
Epoch 2/100 [Train]: 100% 8/8 [00:00<00:00, 66.65it/s]
Epoch 2/100 [Valid]: 100% 2/2 [00:00<00:00, 148.36it/s]
Epoch [02/100] | Train Loss: 0.250243 | Validation Loss: 0.207669
   -> Validation loss cải thiện. Đã lưu model tốt nhất vào 'final_hierarchical_model.pth'
Epoch 3/100 [Train]: 100% 8/8 [00:00<00:00, 86.52it/s]
Epoch 3/100 [Valid]: 100% 2/2 [00:00<00:00, 240.69it/s]
Epoch [03/100] | Train Loss: 0.243089 | Validation Loss: 0.214556
Epoch 4/100 [Train]: 100% 8/8 [00:00<00:00, 87.85it/s]
Epoch 4/100 [Valid]: 100% 2/2 [00:00<00:00, 246.72it/s]
Epoch [04/100] | Train Loss: 0.244731 | Validati

In [ ]:
%%writefile evaluate.py
# evaluate.py

import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

from config import (
    NUM_BASE_FEATURES, NUM_SCALES, LSTM_HIDDEN_UNITS, LSTM_NUM_LAYERS,
    ATTENTION_NUM_HEADS, TRAINED_MODEL_PATH
)
from model import FinalModel
from dataset import get_data_loaders

def run_evaluation():
    """Hàm chính để đánh giá mô hình đa nhiệm trên tập test."""

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Sử dụng thiết bị: {device.upper()}")

    # 1. Tải dữ liệu
    _, test_loader, target_scaler, volatility_scaler = get_data_loaders()
    if not test_loader:
        print("Dừng chương trình vì không tải được dữ liệu.")
        return

    # 2. Tải lại mô hình đã huấn luyện
    print(f"Đang tải mô hình từ: {TRAINED_MODEL_PATH}")
    model = FinalModel(
        input_feature_size=NUM_BASE_FEATURES, num_scales=NUM_SCALES,
        lstm_hidden_units=LSTM_HIDDEN_UNITS, lstm_num_layers=LSTM_NUM_LAYERS,
        num_heads=ATTENTION_NUM_HEADS
    )
    try:
        model.load_state_dict(torch.load(TRAINED_MODEL_PATH, map_location=device))
        model.to(device)
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file model tại '{TRAINED_MODEL_PATH}'. Vui lòng chạy train.py trước.")
        return

    # 3. Đánh giá trên tập Test
    model.eval()
    all_price_preds, all_trend_preds, all_vol_preds = [], [], []
    all_price_lbls, all_trend_lbls, all_vol_lbls = [], [], []

    with torch.no_grad():
        for features, (price_lbl, trend_lbl, vol_lbl) in test_loader:
            features = features.to(device)

            price_pred, trend_pred, vol_pred = model(features)

            # Lưu lại kết quả dự đoán và nhãn thực tế
            all_price_preds.extend(price_pred.cpu().numpy())
            all_trend_preds.extend(trend_pred.cpu().numpy())

            all_price_lbls.extend(price_lbl.numpy())
            all_trend_lbls.extend(trend_lbl.numpy())

    # 4. Xử lý và Giải chuẩn hóa
    # --- Xử lý cho Dự đoán Giá ---
    price_preds = np.array(all_price_preds).reshape(-1, 1)
    price_actuals = np.array(all_price_lbls).reshape(-1, 1)
    original_price_preds = target_scaler.inverse_transform(price_preds)
    original_price_actuals = target_scaler.inverse_transform(price_actuals)

    # --- Xử lý cho Dự đoán Xu hướng ---
    # Chuyển đổi output của model (logits) thành xác suất rồi thành nhãn (0 hoặc 1)
    trend_probs = torch.sigmoid(torch.tensor(all_trend_preds)).numpy()
    trend_preds_labels = (trend_probs > 0.5).astype(int)
    trend_actuals = np.array(all_trend_lbls)

    # 5. Tính toán các chỉ số và in kết quả
    print("\n--- Kết quả Đánh giá trên Tập Test (Model Cuối cùng) ---")

    # --- Nhiệm vụ 1: Dự đoán Giá ---
    mae = np.mean(np.abs(original_price_preds - original_price_actuals))
    print(f"🎯 [Giá] Sai số Trung bình Tuyệt đối (MAE): {mae:.4f} (điểm VN-Index)")

    # --- Nhiệm vụ 2: Dự đoán Xu hướng ---
    accuracy = accuracy_score(trend_actuals, trend_preds_labels)
    print(f"🎯 [Xu hướng] Độ chính xác (Accuracy): {accuracy * 100:.2f}%")

    # 6. Vẽ biểu đồ
    # --- Biểu đồ 1: So sánh Giá ---
    plt.figure(figsize=(15, 6))
    plt.plot(original_price_actuals, label='Giá trị Thực tế', color='blue', marker='.', linestyle='-')
    plt.plot(original_price_preds, label='Giá trị Dự đoán', color='red', linestyle='--')
    plt.title('So sánh Giá trị Thực tế và Dự đoán (Model Cuối cùng)')
    plt.xlabel('Ngày (trong tập Test)')
    plt.ylabel('Giá đóng cửa VN-Index')
    plt.legend()
    plt.grid(True)
    plt.savefig('final_prediction_vs_actual.png')
    print("\n✅ Đã lưu biểu đồ so sánh giá vào file 'final_prediction_vs_actual.png'")

    # --- Biểu đồ 2: Ma trận nhầm lẫn cho Xu hướng ---
    cm = confusion_matrix(trend_actuals, trend_preds_labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Giảm', 'Tăng'])
    fig, ax = plt.subplots(figsize=(6, 6))
    disp.plot(ax=ax, cmap=plt.cm.Blues)
    ax.set_title('Ma trận Nhầm lẫn - Dự đoán Xu hướng')
    plt.savefig('final_confusion_matrix.png')
    print("✅ Đã lưu ma trận nhầm lẫn vào file 'final_confusion_matrix.png'")

if __name__ == '__main__':
    run_evaluation()

Overwriting evaluate.py


In [ ]:
!python evaluate.py

Sử dụng thiết bị: CUDA
✅ DataLoader đa nhiệm đã sẵn sàng.
Đang tải mô hình từ: final_hierarchical_model.pth

--- Kết quả Đánh giá trên Tập Test (Model Cuối cùng) ---
🎯 [Giá] Sai số Trung bình Tuyệt đối (MAE): 18.6659 (điểm VN-Index)
🎯 [Xu hướng] Độ chính xác (Accuracy): 61.29%

✅ Đã lưu biểu đồ so sánh giá vào file 'final_prediction_vs_actual.png'
✅ Đã lưu ma trận nhầm lẫn vào file 'final_confusion_matrix.png'


# ***Bây giờ áp dụng tự động kỹ thuật tối ưu tham số***

In [ ]:
%%writefile model.py
# model.py

import torch
import torch.nn as nn
from config import BATCH_SIZE, LOOKBACK_WINDOW # Chỉ import những gì cần cho việc test

class FinalModel(nn.Module):
    def __init__(self, input_feature_size, num_scales, lstm_hidden_units, lstm_num_layers, num_heads):
        super(FinalModel, self).__init__()

        # SỬA LỖI: Lưu lại các tham số kiến trúc làm thuộc tính của class
        self.input_feature_size = input_feature_size
        self.num_scales = num_scales

        # --- Phần thân chung (Shared Body) ---
        self.lstm_branches = nn.ModuleList([
            nn.LSTM(input_size=self.input_feature_size, hidden_size=lstm_hidden_units,
                    num_layers=lstm_num_layers, batch_first=True, dropout=0.2 if lstm_num_layers > 1 else 0)
            for _ in range(self.num_scales)
        ])
        self.attention = nn.MultiheadAttention(embed_dim=lstm_hidden_units * self.num_scales,
                                             num_heads=num_heads, batch_first=True)

        shared_feature_dim = lstm_hidden_units * self.num_scales
        self.intermediate_layer = nn.Sequential(
            nn.Linear(shared_feature_dim, shared_feature_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.3)
        )

        # --- Các Đầu ra Đa nhiệm (Multi-Task Heads) ---
        self.price_head = nn.Linear(shared_feature_dim // 2, 1)
        self.trend_head = nn.Sequential(
            nn.Linear(shared_feature_dim // 2, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
        self.volatility_head = nn.Linear(shared_feature_dim // 2, 1)

    def forward(self, x):
        branch_outputs = []
        for i in range(self.num_scales):
            # SỬA LỖI: Dùng thuộc tính của class (self.input_feature_size) thay vì biến toàn cục
            start_idx = i * self.input_feature_size
            end_idx = (i + 1) * self.input_feature_size

            branch_input = x[:, :, start_idx:end_idx]
            output, _ = self.lstm_branches[i](branch_input)
            branch_outputs.append(output)

        concatenated_output = torch.cat(branch_outputs, dim=2)
        attention_output, _ = self.attention(concatenated_output, concatenated_output, concatenated_output)
        shared_features = self.intermediate_layer(attention_output[:, -1, :])

        price_prediction = self.price_head(shared_features)
        trend_prediction = self.trend_head(shared_features)
        volatility_prediction = self.volatility_head(shared_features)

        return price_prediction.squeeze(), trend_prediction.squeeze(), volatility_prediction.squeeze()


if __name__ == '__main__':
    # Đoạn test này cần import từ config
    from config import NUM_BASE_FEATURES, NUM_SCALES, LSTM_HIDDEN_UNITS, LSTM_NUM_LAYERS, ATTENTION_NUM_HEADS

    print("--- Kiểm tra kiến trúc Final Model (đã sửa lỗi) ---")
    model = FinalModel(
        input_feature_size=NUM_BASE_FEATURES, num_scales=NUM_SCALES,
        lstm_hidden_units=LSTM_HIDDEN_UNITS, lstm_num_layers=LSTM_NUM_LAYERS,
        num_heads=ATTENTION_NUM_HEADS
    )
    print(model)
    dummy_input = torch.randn(BATCH_SIZE, LOOKBACK_WINDOW, NUM_BASE_FEATURES * NUM_SCALES)
    price, trend, volatility = model(dummy_input)

    print(f"\nShape của input: {dummy_input.shape}")
    print(f"Shape đầu ra Price: {price.shape}")
    print(f"Shape đầu ra Trend: {trend.shape}")
    print(f"Shape đầu ra Volatility: {volatility.shape}")

Overwriting model.py


In [ ]:
%%writefile config.py
# config.py

# --- Đường dẫn file ---
# SỬA LỖI: Trỏ đến đúng file dữ liệu đã được xử lý (có 30 cột)
PROCESSED_DATA_PATH = 'vn_indices_processed.csv'
TRAINED_MODEL_PATH = 'final_optimized_model.pth'

# --- Tham số tạo Dataset ---
LOOKBACK_WINDOW = 30
TARGET_COLUMN = 'VNINDEX_Close'
TEST_SET_SIZE = 0.2
BATCH_SIZE = 32

# --- Tham số Biến đổi Wavelet ---
WAVELET_FAMILY = 'db4'
WAVELET_LEVEL = 4

# --- Tham số Kiến trúc Mặc định ---
NUM_BASE_FEATURES = 30
NUM_SCALES = WAVELET_LEVEL + 1
LSTM_HIDDEN_UNITS = 128
LSTM_NUM_LAYERS = 2
ATTENTION_NUM_HEADS = 4

# --- Tham số cho Multi-Task Learning ---
LOSS_WEIGHTS = {
    'price': 0.6,
    'trend': 0.3,
    'volatility': 0.1
}

# --- Cấu hình cho Optuna ---
OPTUNA_N_TRIALS = 50

Overwriting config.py


In [ ]:
!pip install optuna -q

In [ ]:
%%writefile train_tuner.py
# train_tuner.py

import torch
import torch.nn as nn
import optuna
import numpy as np

from config import (
    NUM_BASE_FEATURES, NUM_SCALES, LOSS_WEIGHTS,
    OPTUNA_N_TRIALS, TRAINED_MODEL_PATH
)
from model import FinalModel
from dataset import get_data_loaders

# 1. Định nghĩa Hàm Mục tiêu (Objective Function)
def objective(trial):
    """
    Hàm này được Optuna gọi trong mỗi lần thử nghiệm.
    Nó sẽ huấn luyện mô hình với bộ tham số do `trial` đề xuất và trả về val_loss.
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # --- Optuna đề xuất các siêu tham số ---
    # `trial.suggest_` sẽ chọn các giá trị trong khoảng cho trước
    lstm_hidden = trial.suggest_int("lstm_hidden_units", 64, 256, step=32)
    lstm_layers = trial.suggest_int("lstm_num_layers", 1, 3)
    attention_heads = trial.suggest_categorical("attention_num_heads", [2, 4, 8])
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)

    # 2. Tải dữ liệu và khởi tạo model với các tham số mới
    train_loader, test_loader, _, _ = get_data_loaders()
    model = FinalModel(
        input_feature_size=NUM_BASE_FEATURES,
        num_scales=NUM_SCALES,
        lstm_hidden_units=lstm_hidden,
        lstm_num_layers=lstm_layers,
        num_heads=attention_heads
    ).to(device)

    price_loss_fn = nn.MSELoss()
    trend_loss_fn = nn.BCEWithLogitsLoss()
    vol_loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Giảm số epoch trong mỗi lần thử để chạy nhanh hơn
    num_epochs = 30
    best_val_loss = float('inf')

    # 3. Vòng lặp huấn luyện rút gọn
    for epoch in range(num_epochs):
        model.train()
        for features, (p_lbl, t_lbl, v_lbl) in train_loader:
            features, p_lbl, t_lbl, v_lbl = features.to(device), p_lbl.to(device), t_lbl.to(device), v_lbl.to(device)
            p_pred, t_pred, v_pred = model(features)
            loss = (LOSS_WEIGHTS['price'] * price_loss_fn(p_pred, p_lbl) +
                    LOSS_WEIGHTS['trend'] * trend_loss_fn(t_pred, t_lbl) +
                    LOSS_WEIGHTS['volatility'] * vol_loss_fn(v_pred, v_lbl))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Đánh giá trên tập validation sau mỗi epoch
        model.eval()
        current_val_loss = 0
        with torch.no_grad():
            for features, (p_lbl, t_lbl, v_lbl) in test_loader:
                features, p_lbl, t_lbl, v_lbl = features.to(device), p_lbl.to(device), t_lbl.to(device), v_lbl.to(device)
                p_pred, t_pred, v_pred = model(features)
                loss = (LOSS_WEIGHTS['price'] * price_loss_fn(p_pred, p_lbl) +
                        LOSS_WEIGHTS['trend'] * trend_loss_fn(t_pred, t_lbl) +
                        LOSS_WEIGHTS['volatility'] * vol_loss_fn(v_pred, v_lbl))
                current_val_loss += loss.item()

        avg_val_loss = current_val_loss / len(test_loader)

        # Báo cáo kết quả cho Optuna
        trial.report(avg_val_loss, epoch)

        # Cắt tỉa (Pruning): Dừng sớm những lần thử không có triển vọng
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss

    # Hàm objective trả về validation loss cuối cùng của lần thử này
    return best_val_loss

# 4. Chạy Cuộc tìm kiếm (Study)
if __name__ == "__main__":
    print("--- Bắt đầu Tối ưu hóa Siêu tham số với Optuna ---")

    # Tạo một "study" với mục tiêu là "minimize" (tối thiểu hóa) hàm objective
    study = optuna.create_study(direction="minimize", pruner=optuna.pruners.MedianPruner())

    # Bắt đầu chạy N_TRIALS lần thử
    study.optimize(objective, n_trials=OPTUNA_N_TRIALS)

    # In ra kết quả
    pruned_trials = study.get_trials(deepcopy=False, states=[optuna.trial.TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[optuna.trial.TrialState.COMPLETE])

    print("\n--- Tối ưu hóa Hoàn tất ---")
    print("Study statistics: ")
    print(f"  Số lần thử hoàn thành: {len(complete_trials)}")
    print(f"  Số lần thử bị cắt tỉa (dừng sớm): {len(pruned_trials)}")

    print("\n🏆 BỘ THAM SỐ TỐT NHẤT:")
    best_params = study.best_trial.params
    for key, value in best_params.items():
        print(f"  {key}: {value}")

    print(f"\nGiá trị Validation Loss tốt nhất: {study.best_value:.6f}")
    print("\n=> Gợi ý: Hãy cập nhật các tham số này vào file config.py và chạy lại file train.py chính thức để huấn luyện mô hình cuối cùng.")

Overwriting train_tuner.py


In [ ]:
!python train_tuner.py

--- Bắt đầu Tối ưu hóa Siêu tham số với Optuna ---
[I 2025-07-09 13:57:29,154] A new study created in memory with name: no-name-5cd36d74-5a33-4460-ba86-d9f0954f3479
✅ DataLoader đa nhiệm đã sẵn sàng.
[I 2025-07-09 13:57:38,735] Trial 0 finished with value: 0.20590250194072723 and parameters: {'lstm_hidden_units': 224, 'lstm_num_layers': 3, 'attention_num_heads': 4, 'learning_rate': 0.0009194168469382918}. Best is trial 0 with value: 0.20590250194072723.
✅ DataLoader đa nhiệm đã sẵn sàng.
[I 2025-07-09 13:57:48,096] Trial 1 finished with value: 0.20616554468870163 and parameters: {'lstm_hidden_units': 256, 'lstm_num_layers': 3, 'attention_num_heads': 2, 'learning_rate': 0.0005738733779054687}. Best is trial 0 with value: 0.20590250194072723.
✅ DataLoader đa nhiệm đã sẵn sàng.
[I 2025-07-09 13:57:54,001] Trial 2 finished with value: 0.20579843968153 and parameters: {'lstm_hidden_units': 192, 'lstm_num_layers': 1, 'attention_num_heads': 8, 'learning_rate': 0.0022088373370435133}. Best is 

# ***Giờ test thử siêu tham số mới***

In [ ]:
%%writefile model.py
# model.py

import torch
import torch.nn as nn
from config import BATCH_SIZE, LOOKBACK_WINDOW # Chỉ import những gì cần cho việc test

class FinalModel(nn.Module):
    def __init__(self, input_feature_size, num_scales, lstm_hidden_units, lstm_num_layers, num_heads):
        super(FinalModel, self).__init__()

        # SỬA LỖI: Lưu lại các tham số kiến trúc làm thuộc tính của class
        self.input_feature_size = input_feature_size
        self.num_scales = num_scales

        # --- Phần thân chung (Shared Body) ---
        self.lstm_branches = nn.ModuleList([
            nn.LSTM(input_size=self.input_feature_size, hidden_size=lstm_hidden_units,
                    num_layers=lstm_num_layers, batch_first=True, dropout=0.2 if lstm_num_layers > 1 else 0)
            for _ in range(self.num_scales)
        ])
        self.attention = nn.MultiheadAttention(embed_dim=lstm_hidden_units * self.num_scales,
                                             num_heads=num_heads, batch_first=True)

        shared_feature_dim = lstm_hidden_units * self.num_scales
        self.intermediate_layer = nn.Sequential(
            nn.Linear(shared_feature_dim, shared_feature_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.3)
        )

        # --- Các Đầu ra Đa nhiệm (Multi-Task Heads) ---
        self.price_head = nn.Linear(shared_feature_dim // 2, 1)
        self.trend_head = nn.Sequential(
            nn.Linear(shared_feature_dim // 2, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
        self.volatility_head = nn.Linear(shared_feature_dim // 2, 1)

    def forward(self, x):
        branch_outputs = []
        for i in range(self.num_scales):
            # SỬA LỖI: Dùng thuộc tính của class (self.input_feature_size) thay vì biến toàn cục
            start_idx = i * self.input_feature_size
            end_idx = (i + 1) * self.input_feature_size

            branch_input = x[:, :, start_idx:end_idx]
            output, _ = self.lstm_branches[i](branch_input)
            branch_outputs.append(output)

        concatenated_output = torch.cat(branch_outputs, dim=2)
        attention_output, _ = self.attention(concatenated_output, concatenated_output, concatenated_output)
        shared_features = self.intermediate_layer(attention_output[:, -1, :])

        price_prediction = self.price_head(shared_features)
        trend_prediction = self.trend_head(shared_features)
        volatility_prediction = self.volatility_head(shared_features)

        return price_prediction.squeeze(), trend_prediction.squeeze(), volatility_prediction.squeeze()


if __name__ == '__main__':
    # Đoạn test này cần import từ config
    from config import NUM_BASE_FEATURES, NUM_SCALES, LSTM_HIDDEN_UNITS, LSTM_NUM_LAYERS, ATTENTION_NUM_HEADS

    print("--- Kiểm tra kiến trúc Final Model (đã sửa lỗi) ---")
    model = FinalModel(
        input_feature_size=NUM_BASE_FEATURES, num_scales=NUM_SCALES,
        lstm_hidden_units=LSTM_HIDDEN_UNITS, lstm_num_layers=LSTM_NUM_LAYERS,
        num_heads=ATTENTION_NUM_HEADS
    )
    print(model)
    dummy_input = torch.randn(BATCH_SIZE, LOOKBACK_WINDOW, NUM_BASE_FEATURES * NUM_SCALES)
    price, trend, volatility = model(dummy_input)

    print(f"\nShape của input: {dummy_input.shape}")
    print(f"Shape đầu ra Price: {price.shape}")
    print(f"Shape đầu ra Trend: {trend.shape}")
    print(f"Shape đầu ra Volatility: {volatility.shape}")

Overwriting model.py


In [ ]:
!python model.py

--- Kiểm tra kiến trúc Final Model (đã sửa lỗi) ---
FinalModel(
  (lstm_branches): ModuleList(
    (0-4): 5 x LSTM(30, 128, num_layers=2, batch_first=True, dropout=0.2)
  )
  (attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=640, out_features=640, bias=True)
  )
  (intermediate_layer): Sequential(
    (0): Linear(in_features=640, out_features=320, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
  )
  (price_head): Linear(in_features=320, out_features=1, bias=True)
  (trend_head): Sequential(
    (0): Linear(in_features=320, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=1, bias=True)
  )
  (volatility_head): Linear(in_features=320, out_features=1, bias=True)
)

Shape của input: torch.Size([32, 30, 150])
Shape đầu ra Price: torch.Size([32])
Shape đầu ra Trend: torch.Size([32])
Shape đầu ra Volatility: torch.Size([32])


In [ ]:
%%writefile config.py
# config.py

# --- Đường dẫn file ---
PROCESSED_DATA_PATH = 'vn_indices_processed.csv'
# File model cuối cùng sau khi đã được tối ưu
TRAINED_MODEL_PATH = 'final_optimized_model.pth'

# --- Tham số tạo Dataset ---
LOOKBACK_WINDOW = 30
TARGET_COLUMN = 'VNINDEX_Close'
TEST_SET_SIZE = 0.2
BATCH_SIZE = 32

# --- Tham số Biến đổi Wavelet ---
WAVELET_FAMILY = 'db4'
WAVELET_LEVEL = 4

# --- THAM SỐ KIẾN TRÚC TỐT NHẤT TỪ OPTUNA ---
NUM_BASE_FEATURES = 30
NUM_SCALES = WAVELET_LEVEL + 1
LSTM_HIDDEN_UNITS = 192   # <--- CẬP NHẬT
LSTM_NUM_LAYERS = 1       # <--- CẬP NHẬT
ATTENTION_NUM_HEADS = 8   # <--- CẬP NHẬT

# --- Tham số cho Multi-Task Learning ---
LOSS_WEIGHTS = {
    'price': 0.6,
    'trend': 0.3,
    'volatility': 0.1
}

# --- Tham số Huấn luyện TỐT NHẤT TỪ OPTUNA ---
LEARNING_RATE = 0.0022088373370435133 # <--- CẬP NHẬT
NUM_EPOCHS = 100 # Giữ nguyên 100 epoch để học sâu hơn

Overwriting config.py


In [ ]:
%%writefile train.py
# train.py

import torch
import torch.nn as nn
from tqdm import tqdm

from config import (
    NUM_BASE_FEATURES, NUM_SCALES, LSTM_HIDDEN_UNITS, LSTM_NUM_LAYERS,
    ATTENTION_NUM_HEADS, LOSS_WEIGHTS,
    LEARNING_RATE, NUM_EPOCHS, TRAINED_MODEL_PATH
)
from model import FinalModel
from dataset import get_data_loaders

def run_training():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Sử dụng thiết bị: {device.upper()}")

    train_loader, test_loader, _, _ = get_data_loaders()
    if not train_loader: return

    model = FinalModel(
        input_feature_size=NUM_BASE_FEATURES, num_scales=NUM_SCALES,
        lstm_hidden_units=LSTM_HIDDEN_UNITS, lstm_num_layers=LSTM_NUM_LAYERS,
        num_heads=ATTENTION_NUM_HEADS
    ).to(device)

    # Định nghĩa các hàm loss cho từng nhiệm vụ
    price_loss_fn = nn.MSELoss() # Cho giá (hồi quy)
    # Dùng BCEWithLogitsLoss cho độ ổn định số học, nó đã tích hợp sẵn Sigmoid
    trend_loss_fn = nn.BCEWithLogitsLoss() # Cho xu hướng (phân loại)
    vol_loss_fn = nn.MSELoss() # Cho độ biến động (hồi quy)

    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

    print("\n--- Bắt đầu Huấn luyện Model Cuối cùng (Hierarchical + Multi-Task) ---")
    best_val_loss = float('inf')

    for epoch in range(NUM_EPOCHS):
        # --- PHA HUẤN LUYỆN ---
        model.train()
        total_train_loss = 0
        for features, (price_lbl, trend_lbl, vol_lbl) in tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]"):
            features = features.to(device)
            price_lbl, trend_lbl, vol_lbl = price_lbl.to(device), trend_lbl.to(device), vol_lbl.to(device)

            # Lấy 3 đầu ra từ model
            price_pred, trend_pred, vol_pred = model(features)

            # Tính toán loss cho từng nhiệm vụ
            loss_p = price_loss_fn(price_pred, price_lbl)
            loss_t = trend_loss_fn(trend_pred, trend_lbl)
            loss_v = vol_loss_fn(vol_pred, vol_lbl)

            # Tính loss tổng hợp dựa trên trọng số
            total_loss = (LOSS_WEIGHTS['price'] * loss_p +
                          LOSS_WEIGHTS['trend'] * loss_t +
                          LOSS_WEIGHTS['volatility'] * loss_v)

            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()
            total_train_loss += total_loss.item()

        # --- PHA KIỂM ĐỊNH ---
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for features, (price_lbl, trend_lbl, vol_lbl) in tqdm(test_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Valid]"):
                features = features.to(device)
                price_lbl, trend_lbl, vol_lbl = price_lbl.to(device), trend_lbl.to(device), vol_lbl.to(device)

                price_pred, trend_pred, vol_pred = model(features)

                loss_p = price_loss_fn(price_pred, price_lbl)
                loss_t = trend_loss_fn(trend_pred, trend_lbl)
                loss_v = vol_loss_fn(vol_pred, vol_lbl)

                total_loss = (LOSS_WEIGHTS['price'] * loss_p +
                              LOSS_WEIGHTS['trend'] * loss_t +
                              LOSS_WEIGHTS['volatility'] * loss_v)
                total_val_loss += total_loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        avg_val_loss = total_val_loss / len(test_loader)

        print(f"Epoch [{epoch+1:02d}/{NUM_EPOCHS}] | Train Loss: {avg_train_loss:.6f} | Validation Loss: {avg_val_loss:.6f}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), TRAINED_MODEL_PATH)
            print(f"   -> Validation loss cải thiện. Đã lưu model tốt nhất vào '{TRAINED_MODEL_PATH}'")

    print(f"\n--- Huấn luyện Hoàn tất ---")
    print(f"✅ Model cuối cùng đã được lưu tại epoch có Validation Loss = {best_val_loss:.6f}")

if __name__ == '__main__':
    run_training()

Overwriting train.py


In [ ]:
!python train.py

Sử dụng thiết bị: CUDA
✅ DataLoader đa nhiệm đã sẵn sàng.

--- Bắt đầu Huấn luyện Model Cuối cùng (Hierarchical + Multi-Task) ---
Epoch 1/100 [Train]: 100% 8/8 [00:00<00:00, 17.59it/s]
Epoch 1/100 [Valid]: 100% 2/2 [00:00<00:00, 73.48it/s]
Epoch [01/100] | Train Loss: 0.445531 | Validation Loss: 0.222070
   -> Validation loss cải thiện. Đã lưu model tốt nhất vào 'final_optimized_model.pth'
Epoch 2/100 [Train]: 100% 8/8 [00:00<00:00, 43.60it/s]
Epoch 2/100 [Valid]: 100% 2/2 [00:00<00:00, 77.30it/s]
Epoch [02/100] | Train Loss: 0.273318 | Validation Loss: 0.207194
   -> Validation loss cải thiện. Đã lưu model tốt nhất vào 'final_optimized_model.pth'
Epoch 3/100 [Train]: 100% 8/8 [00:00<00:00, 45.33it/s]
Epoch 3/100 [Valid]: 100% 2/2 [00:00<00:00, 77.78it/s]
Epoch [03/100] | Train Loss: 0.245420 | Validation Loss: 0.220386
Epoch 4/100 [Train]: 100% 8/8 [00:00<00:00, 45.12it/s]
Epoch 4/100 [Valid]: 100% 2/2 [00:00<00:00, 75.73it/s]
Epoch [04/100] | Train Loss: 0.253762 | Validation Loss: 0

In [ ]:
%%writefile evaluate.py
# evaluate.py

import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

from config import (
    NUM_BASE_FEATURES, NUM_SCALES, LSTM_HIDDEN_UNITS, LSTM_NUM_LAYERS,
    ATTENTION_NUM_HEADS, TRAINED_MODEL_PATH
)
from model import FinalModel
from dataset import get_data_loaders

def run_evaluation():
    """Hàm chính để đánh giá mô hình đa nhiệm trên tập test."""

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Sử dụng thiết bị: {device.upper()}")

    # 1. Tải dữ liệu
    _, test_loader, target_scaler, volatility_scaler = get_data_loaders()
    if not test_loader:
        print("Dừng chương trình vì không tải được dữ liệu.")
        return

    # 2. Tải lại mô hình đã huấn luyện
    print(f"Đang tải mô hình từ: {TRAINED_MODEL_PATH}")
    model = FinalModel(
        input_feature_size=NUM_BASE_FEATURES, num_scales=NUM_SCALES,
        lstm_hidden_units=LSTM_HIDDEN_UNITS, lstm_num_layers=LSTM_NUM_LAYERS,
        num_heads=ATTENTION_NUM_HEADS
    )
    try:
        model.load_state_dict(torch.load(TRAINED_MODEL_PATH, map_location=device))
        model.to(device)
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file model tại '{TRAINED_MODEL_PATH}'. Vui lòng chạy train.py trước.")
        return

    # 3. Đánh giá trên tập Test
    model.eval()
    all_price_preds, all_trend_preds, all_vol_preds = [], [], []
    all_price_lbls, all_trend_lbls, all_vol_lbls = [], [], []

    with torch.no_grad():
        for features, (price_lbl, trend_lbl, vol_lbl) in test_loader:
            features = features.to(device)

            price_pred, trend_pred, vol_pred = model(features)

            # Lưu lại kết quả dự đoán và nhãn thực tế
            all_price_preds.extend(price_pred.cpu().numpy())
            all_trend_preds.extend(trend_pred.cpu().numpy())

            all_price_lbls.extend(price_lbl.numpy())
            all_trend_lbls.extend(trend_lbl.numpy())

    # 4. Xử lý và Giải chuẩn hóa
    # --- Xử lý cho Dự đoán Giá ---
    price_preds = np.array(all_price_preds).reshape(-1, 1)
    price_actuals = np.array(all_price_lbls).reshape(-1, 1)
    original_price_preds = target_scaler.inverse_transform(price_preds)
    original_price_actuals = target_scaler.inverse_transform(price_actuals)

    # --- Xử lý cho Dự đoán Xu hướng ---
    # Chuyển đổi output của model (logits) thành xác suất rồi thành nhãn (0 hoặc 1)
    trend_probs = torch.sigmoid(torch.tensor(all_trend_preds)).numpy()
    trend_preds_labels = (trend_probs > 0.5).astype(int)
    trend_actuals = np.array(all_trend_lbls)

    # 5. Tính toán các chỉ số và in kết quả
    print("\n--- Kết quả Đánh giá trên Tập Test (Model Cuối cùng) ---")

    # --- Nhiệm vụ 1: Dự đoán Giá ---
    mae = np.mean(np.abs(original_price_preds - original_price_actuals))
    print(f"🎯 [Giá] Sai số Trung bình Tuyệt đối (MAE): {mae:.4f} (điểm VN-Index)")

    # --- Nhiệm vụ 2: Dự đoán Xu hướng ---
    accuracy = accuracy_score(trend_actuals, trend_preds_labels)
    print(f"🎯 [Xu hướng] Độ chính xác (Accuracy): {accuracy * 100:.2f}%")

    # 6. Vẽ biểu đồ
    # --- Biểu đồ 1: So sánh Giá ---
    plt.figure(figsize=(15, 6))
    plt.plot(original_price_actuals, label='Giá trị Thực tế', color='blue', marker='.', linestyle='-')
    plt.plot(original_price_preds, label='Giá trị Dự đoán', color='red', linestyle='--')
    plt.title('So sánh Giá trị Thực tế và Dự đoán (Model Cuối cùng)')
    plt.xlabel('Ngày (trong tập Test)')
    plt.ylabel('Giá đóng cửa VN-Index')
    plt.legend()
    plt.grid(True)
    plt.savefig('final_prediction_vs_actual.png')
    print("\n✅ Đã lưu biểu đồ so sánh giá vào file 'final_prediction_vs_actual.png'")

    # --- Biểu đồ 2: Ma trận nhầm lẫn cho Xu hướng ---
    cm = confusion_matrix(trend_actuals, trend_preds_labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Giảm', 'Tăng'])
    fig, ax = plt.subplots(figsize=(6, 6))
    disp.plot(ax=ax, cmap=plt.cm.Blues)
    ax.set_title('Ma trận Nhầm lẫn - Dự đoán Xu hướng')
    plt.savefig('final_confusion_matrix.png')
    print("✅ Đã lưu ma trận nhầm lẫn vào file 'final_confusion_matrix.png'")

if __name__ == '__main__':
    run_evaluation()

Overwriting evaluate.py


In [ ]:
!python evaluate.py

Sử dụng thiết bị: CUDA
✅ DataLoader đa nhiệm đã sẵn sàng.
Đang tải mô hình từ: final_optimized_model.pth

--- Kết quả Đánh giá trên Tập Test (Model Cuối cùng) ---
🎯 [Giá] Sai số Trung bình Tuyệt đối (MAE): 20.7005 (điểm VN-Index)
🎯 [Xu hướng] Độ chính xác (Accuracy): 61.29%

✅ Đã lưu biểu đồ so sánh giá vào file 'final_prediction_vs_actual.png'
✅ Đã lưu ma trận nhầm lẫn vào file 'final_confusion_matrix.png'


# ***ESEMBLE***

In [ ]:
%%writefile model.py
# model.py

import torch
import torch.nn as nn
from config import BATCH_SIZE, LOOKBACK_WINDOW, NUM_BASE_FEATURES, NUM_SCALES

# --- KIẾN TRÚC 1: MS-LSTM + ATTENTION ---
class MSLSTMAttention(nn.Module):
    def __init__(self, input_feature_size, num_scales, lstm_hidden_units, lstm_num_layers, num_heads):
        super(MSLSTMAttention, self).__init__()
        self.num_scales = num_scales
        self.input_feature_size = input_feature_size
        self.lstm_branches = nn.ModuleList([
            nn.LSTM(input_size=input_feature_size, hidden_size=lstm_hidden_units,
                    num_layers=lstm_num_layers, batch_first=True, dropout=0.2 if lstm_num_layers > 1 else 0)
            for _ in range(num_scales)
        ])
        self.attention = nn.MultiheadAttention(
            embed_dim=lstm_hidden_units * num_scales,
            num_heads=num_heads,
            batch_first=True
        )
        self.fc = nn.Linear(in_features=lstm_hidden_units * num_scales, out_features=1)

    def forward(self, x):
        branch_outputs = []
        for i in range(self.num_scales):
            branch_input = x[:, :, i*self.input_feature_size : (i+1)*self.input_feature_size]
            output, _ = self.lstm_branches[i](branch_input)
            branch_outputs.append(output)

        concatenated_output = torch.cat(branch_outputs, dim=2)
        attention_output, _ = self.attention(concatenated_output, concatenated_output, concatenated_output)
        last_time_step_output = attention_output[:, -1, :]
        final_prediction = self.fc(last_time_step_output)
        return final_prediction.squeeze()

# --- KIẾN TRÚC 2: FINAL MODEL (HIERARCHICAL + MULTI-TASK) ---
class FinalModel(nn.Module):
    def __init__(self, input_feature_size, num_scales, lstm_hidden_units, lstm_num_layers, num_heads):
        super(FinalModel, self).__init__()
        self.input_feature_size = input_feature_size
        self.num_scales = num_scales

        self.lstm_branches = nn.ModuleList([
            nn.LSTM(input_size=self.input_feature_size, hidden_size=lstm_hidden_units,
                    num_layers=lstm_num_layers, batch_first=True, dropout=0.2 if lstm_num_layers > 1 else 0)
            for _ in range(self.num_scales)
        ])
        self.attention = nn.MultiheadAttention(embed_dim=lstm_hidden_units * self.num_scales,
                                             num_heads=num_heads, batch_first=True)

        shared_feature_dim = lstm_hidden_units * self.num_scales
        self.intermediate_layer = nn.Sequential(
            nn.Linear(shared_feature_dim, shared_feature_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.3)
        )

        self.price_head = nn.Linear(shared_feature_dim // 2, 1)
        self.trend_head = nn.Sequential(
            nn.Linear(shared_feature_dim // 2, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
        self.volatility_head = nn.Linear(shared_feature_dim // 2, 1)

    def forward(self, x):
        branch_outputs = []
        for i in range(self.num_scales):
            branch_input = x[:, :, i*self.input_feature_size : (i+1)*self.input_feature_size]
            output, _ = self.lstm_branches[i](branch_input)
            branch_outputs.append(output)

        concatenated_output = torch.cat(branch_outputs, dim=2)
        attention_output, _ = self.attention(concatenated_output, concatenated_output, concatenated_output)
        shared_features = self.intermediate_layer(attention_output[:, -1, :])

        price_prediction = self.price_head(shared_features)
        trend_prediction = self.trend_head(shared_features)
        volatility_prediction = self.volatility_head(shared_features)

        return price_prediction.squeeze(), trend_prediction.squeeze(), volatility_prediction.squeeze()

Writing model.py


In [ ]:
!python model.py

In [ ]:
%%writefile ensemble_evaluate.py
# ensemble_evaluate.py

import torch
import numpy as np
import matplotlib.pyplot as plt

# Import cả hai lớp kiến trúc
from model import MSLSTMAttention, FinalModel
from dataset import get_data_loaders
from config import NUM_BASE_FEATURES, NUM_SCALES

def run_ensemble_evaluation():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Sử dụng thiết bị: {device.upper()}")

    # 1. Tải dữ liệu
    # Chúng ta cần target_scaler để giải chuẩn hóa
    _, test_loader, target_scaler, _ = get_data_loaders()
    if not test_loader: return

    # 2. Định nghĩa và tải các model
    # (Giữ nguyên phần tải model)
    model_A_params = {'lstm_hidden_units': 128, 'lstm_num_layers': 2, 'attention_num_heads': 4}
    model_A_path = 'mslstm_attention_tuned.pth'
    model_A = MSLSTMAttention(
        input_feature_size=NUM_BASE_FEATURES, num_scales=NUM_SCALES,
        lstm_hidden_units=model_A_params['lstm_hidden_units'],
        lstm_num_layers=model_A_params['lstm_num_layers'],
        num_heads=model_A_params['attention_num_heads']
    )

    model_B_params = {'lstm_hidden_units': 192, 'lstm_num_layers': 1, 'attention_num_heads': 8}
    model_B_path = 'final_optimized_model.pth'
    model_B = FinalModel(
        input_feature_size=NUM_BASE_FEATURES, num_scales=NUM_SCALES,
        lstm_hidden_units=model_B_params['lstm_hidden_units'],
        lstm_num_layers=model_B_params['lstm_num_layers'],
        num_heads=model_B_params['attention_num_heads']
    )

    try:
        print(f"Đang tải Model A từ: {model_A_path}")
        model_A.load_state_dict(torch.load(model_A_path, map_location=device))
        model_A.to(device)
        model_A.eval()

        print(f"Đang tải Model B từ: {model_B_path}")
        model_B.load_state_dict(torch.load(model_B_path, map_location=device))
        model_B.to(device)
        model_B.eval()
    except FileNotFoundError as e:
        print(f"Lỗi: Không tìm thấy file model. Hãy đảm bảo các file sau tồn tại: {e.filename}")
        return

    # 3. Lấy dự đoán và thực hiện ensemble
    original_preds_ensemble = []
    original_actuals_list = []

    with torch.no_grad():
        for features, (price_lbl, _, _) in test_loader:
            features = features.to(device)

            # --- LOGIC SỬA LỖI ---
            # a. Lấy dự đoán đã chuẩn hóa từ mỗi model
            scaled_pred_A = model_A(features)
            scaled_pred_B_price, _, _ = model_B(features)

            # b. Giải chuẩn hóa từng dự đoán một
            original_pred_A = target_scaler.inverse_transform(scaled_pred_A.cpu().numpy().reshape(-1, 1))
            original_pred_B = target_scaler.inverse_transform(scaled_pred_B_price.cpu().numpy().reshape(-1, 1))

            # c. Lấy trung bình các dự đoán đã được giải chuẩn hóa
            avg_pred_original = (original_pred_A + original_pred_B) / 2.0

            # d. Giải chuẩn hóa giá trị thực để so sánh
            original_actuals = target_scaler.inverse_transform(price_lbl.numpy().reshape(-1, 1))

            original_preds_ensemble.extend(avg_pred_original.flatten())
            original_actuals_list.extend(original_actuals.flatten())

    # 4. Tính toán sai số
    mae = np.mean(np.abs(np.array(original_preds_ensemble) - np.array(original_actuals_list)))
    print("\n--- Kết quả Đánh giá của 'Hội đồng Chuyên gia' (Ensemble) - ĐÃ SỬA LỖI ---")
    print(f"🏆 [Giá] Sai số Trung bình Tuyệt đối (MAE): {mae:.4f} (điểm VN-Index)")

    # 5. Vẽ biểu đồ
    plt.figure(figsize=(15, 6))
    plt.plot(original_actuals_list, label='Giá trị Thực tế', color='blue', marker='.', linestyle='-')
    plt.plot(original_preds_ensemble, label='Dự đoán Ensemble', color='green', linestyle='--')
    plt.title("So sánh Giá trị Thực tế và Dự đoán Ensemble (Đã sửa lỗi)")
    plt.xlabel('Ngày (trong tập Test)')
    plt.ylabel('Giá đóng cửa VN-Index')
    plt.legend()
    plt.grid(True)
    plt.savefig('final_ensemble_prediction_corrected.png')
    print("✅ Đã lưu biểu đồ so sánh của Ensemble vào file 'final_ensemble_prediction_corrected.png'")

if __name__ == '__main__':
    run_ensemble_evaluation()

Overwriting ensemble_evaluate.py


In [ ]:
!python ensemble_evaluate.py

Sử dụng thiết bị: CUDA
✅ DataLoader đa nhiệm đã sẵn sàng.
Đang tải Model A từ: mslstm_attention_tuned.pth
Đang tải Model B từ: final_optimized_model.pth

--- Kết quả Đánh giá của 'Hội đồng Chuyên gia' (Ensemble) - ĐÃ SỬA LỖI ---
🏆 [Giá] Sai số Trung bình Tuyệt đối (MAE): 33.4388 (điểm VN-Index)
✅ Đã lưu biểu đồ so sánh của Ensemble vào file 'final_ensemble_prediction_corrected.png'


In [ ]:
%%writefile train_simple_model.py
# train_simple_model.py

import torch
import torch.nn as nn
from tqdm import tqdm
import numpy as np

# Import các config đã được tối ưu từ bước trước
from config import (
    NUM_BASE_FEATURES, NUM_SCALES,
    # Dùng các tham số mặc định nhưng hiệu quả cho model đơn giản hơn
    LSTM_HIDDEN_UNITS, LSTM_NUM_LAYERS, ATTENTION_NUM_HEADS,
    LEARNING_RATE, NUM_EPOCHS
)
# Import kiến trúc model đơn giản
from model import MSLSTMAttention
# QUAN TRỌNG: Import hàm get_data_loaders mới nhất để đảm bảo đồng bộ
from dataset import get_data_loaders

# Đặt tên file riêng cho model này để tránh ghi đè
SIMPLE_MODEL_PATH = 'mslstm_attention_synced.pth'

def run_simple_training():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Sử dụng thiết bị: {device.upper()}")

    # 1. Tải dữ liệu bằng data loader mới nhất
    # Nó sẽ trả về 3 loại label, nhưng chúng ta chỉ dùng price_label
    train_loader, test_loader, _, _ = get_data_loaders()
    if not train_loader: return

    # 2. Khởi tạo Model A
    model = MSLSTMAttention(
        input_feature_size=NUM_BASE_FEATURES,
        num_scales=NUM_SCALES,
        lstm_hidden_units=128, # Dùng một giá trị hợp lý
        lstm_num_layers=2,
        num_heads=4
    ).to(device)

    # Model này chỉ có 1 nhiệm vụ nên chỉ cần 1 hàm loss
    loss_function = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    print("\n--- Bắt đầu Huấn luyện lại Model A (Đồng bộ) ---")
    best_val_loss = float('inf')

    for epoch in range(NUM_EPOCHS):
        model.train()
        train_loss = 0.0
        # Chỉ lấy price_lbl từ bộ dữ liệu
        for features, (price_lbl, _, _) in tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]"):
            features, price_lbl = features.to(device), price_lbl.to(device)

            outputs = model(features)
            loss = loss_function(outputs, price_lbl)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for features, (price_lbl, _, _) in test_loader:
                features, price_lbl = features.to(device), price_lbl.to(device)
                outputs = model(features)
                loss = loss_function(outputs, price_lbl)
                val_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(test_loader)

        print(f"Epoch [{epoch+1:02d}/{NUM_EPOCHS}] | Train Loss: {avg_train_loss:.6f} | Val Loss: {avg_val_loss:.6f}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), SIMPLE_MODEL_PATH)
            print(f"   -> Val loss cải thiện. Đã lưu model vào '{SIMPLE_MODEL_PATH}'")

    print(f"\n--- Huấn luyện Hoàn tất ---")

if __name__ == '__main__':
    run_simple_training()

Writing train_simple_model.py


In [ ]:
!python train_simple_model.py

Sử dụng thiết bị: CUDA
✅ DataLoader đa nhiệm đã sẵn sàng.

--- Bắt đầu Huấn luyện lại Model A (Đồng bộ) ---
Epoch 1/100 [Train]: 100% 8/8 [00:00<00:00, 22.98it/s]
Epoch [01/100] | Train Loss: 0.129956 | Val Loss: 0.006859
   -> Val loss cải thiện. Đã lưu model vào 'mslstm_attention_synced.pth'
Epoch 2/100 [Train]: 100% 8/8 [00:00<00:00, 74.54it/s]
Epoch [02/100] | Train Loss: 0.052934 | Val Loss: 0.036107
Epoch 3/100 [Train]: 100% 8/8 [00:00<00:00, 90.49it/s]
Epoch [03/100] | Train Loss: 0.048240 | Val Loss: 0.004623
   -> Val loss cải thiện. Đã lưu model vào 'mslstm_attention_synced.pth'
Epoch 4/100 [Train]: 100% 8/8 [00:00<00:00, 97.95it/s]
Epoch [04/100] | Train Loss: 0.042537 | Val Loss: 0.009207
Epoch 5/100 [Train]: 100% 8/8 [00:00<00:00, 105.60it/s]
Epoch [05/100] | Train Loss: 0.042286 | Val Loss: 0.008672
Epoch 6/100 [Train]: 100% 8/8 [00:00<00:00, 106.50it/s]
Epoch [06/100] | Train Loss: 0.044165 | Val Loss: 0.006925
Epoch 7/100 [Train]: 100% 8/8 [00:00<00:00, 105.92it/s]
Epoc

In [ ]:
%%writefile ensemble_evaluate.py
# ensemble_evaluate.py

import torch
import numpy as np
import matplotlib.pyplot as plt

from model import MSLSTMAttention, FinalModel
from dataset import get_data_loaders
from config import NUM_BASE_FEATURES, NUM_SCALES

# Các đường dẫn file model mới
SYNCED_MODEL_A_PATH = 'mslstm_attention_synced.pth'
OPTIMIZED_MODEL_B_PATH = 'final_optimized_model.pth'

def run_ensemble_evaluation():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Sử dụng thiết bị: {device.upper()}")

    _, test_loader, target_scaler, _ = get_data_loaders()
    if not test_loader: return

    # Model A (đã được huấn luyện đồng bộ)
    model_A = MSLSTMAttention(
        input_feature_size=NUM_BASE_FEATURES, num_scales=NUM_SCALES,
        lstm_hidden_units=128, lstm_num_layers=2, num_heads=4
    )

    # Model B (tối ưu)
    model_B_params = {'lstm_hidden_units': 192, 'lstm_num_layers': 1, 'attention_num_heads': 8}
    model_B = FinalModel(
        input_feature_size=NUM_BASE_FEATURES, num_scales=NUM_SCALES,
        lstm_hidden_units=model_B_params['lstm_hidden_units'],
        lstm_num_layers=model_B_params['lstm_num_layers'],
        num_heads=model_B_params['attention_num_heads']
    )

    try:
        print(f"Đang tải Model A (đồng bộ) từ: {SYNCED_MODEL_A_PATH}")
        model_A.load_state_dict(torch.load(SYNCED_MODEL_A_PATH, map_location=device))
        model_A.to(device)
        model_A.eval()

        print(f"Đang tải Model B (tối ưu) từ: {OPTIMIZED_MODEL_B_PATH}")
        model_B.load_state_dict(torch.load(OPTIMIZED_MODEL_B_PATH, map_location=device))
        model_B.to(device)
        model_B.eval()
    except FileNotFoundError as e:
        print(f"Lỗi: Không tìm thấy file model. Hãy đảm bảo các file sau tồn tại: {e.filename}")
        return

    # Lấy dự đoán và thực hiện ensemble (logic giải chuẩn hóa trước đã đúng)
    original_preds_ensemble = []
    original_actuals_list = []

    with torch.no_grad():
        for features, (price_lbl, _, _) in test_loader:
            features = features.to(device)

            scaled_pred_A = model_A(features)
            scaled_pred_B_price, _, _ = model_B(features)

            original_pred_A = target_scaler.inverse_transform(scaled_pred_A.cpu().numpy().reshape(-1, 1))
            original_pred_B = target_scaler.inverse_transform(scaled_pred_B_price.cpu().numpy().reshape(-1, 1))

            avg_pred_original = (original_pred_A + original_pred_B) / 2.0

            original_actuals = target_scaler.inverse_transform(price_lbl.numpy().reshape(-1, 1))

            original_preds_ensemble.extend(avg_pred_original.flatten())
            original_actuals_list.extend(original_actuals.flatten())

    mae = np.mean(np.abs(np.array(original_preds_ensemble) - np.array(original_actuals_list)))
    print("\n--- Kết quả Đánh giá của Ensemble (Đã đồng bộ dữ liệu) ---")
    print(f"🏆 [Giá] Sai số Trung bình Tuyệt đối (MAE): {mae:.4f} (điểm VN-Index)")

    # Vẽ biểu đồ
    plt.figure(figsize=(15, 6))
    plt.plot(original_actuals_list, label='Giá trị Thực tế', color='blue', marker='.', linestyle='-')
    plt.plot(original_preds_ensemble, label='Dự đoán Ensemble', color='green', linestyle='--')
    plt.title("So sánh Giá trị Thực tế và Dự đoán Ensemble (Đồng bộ)")
    plt.xlabel('Ngày (trong tập Test)')
    plt.ylabel('Giá đóng cửa VN-Index')
    plt.legend()
    plt.grid(True)
    plt.savefig('final_ensemble_prediction_synced.png')
    print("✅ Đã lưu biểu đồ so sánh của Ensemble vào file 'final_ensemble_prediction_synced.png'")

if __name__ == '__main__':
    run_ensemble_evaluation()

Overwriting ensemble_evaluate.py


In [ ]:
!python ensemble_evaluate.py

Sử dụng thiết bị: CUDA
✅ DataLoader đa nhiệm đã sẵn sàng.
Đang tải Model A (đồng bộ) từ: mslstm_attention_synced.pth
Đang tải Model B (tối ưu) từ: final_optimized_model.pth

--- Kết quả Đánh giá của Ensemble (Đã đồng bộ dữ liệu) ---
🏆 [Giá] Sai số Trung bình Tuyệt đối (MAE): 19.0011 (điểm VN-Index)
✅ Đã lưu biểu đồ so sánh của Ensemble vào file 'final_ensemble_prediction_synced.png'
